### **_Importar librerias_**

In [1209]:
# Principales
import pandas as pd
import numpy as np

# Modificaciones 
import re
from unidecode import unidecode
import unicodedata
import datetime


# visuzalizaciones
import missingno as msno
import matplotlib as plt 
import plotly.express as px 

In [1210]:
pd.set_option("display.max_columns",None)

pd.set_option('display.max_colwidth', None)

### **_Lectura de datos_**

In [1211]:
datos_pisosmadrid_normal= pd.read_excel("../01-datos-iniciales/04-PisosMadrid/PisosMadrid.xlsx")
datos_pisosmadrid_detalles= pd.read_excel("../01-datos-iniciales/04-PisosMadrid/Detalle_pisosmadrid.xlsx")
dfbarrios= pd.read_excel('../02-datos-limpios/distr_barrios_codpostal.xlsx')

dfcalles= pd.read_excel('../02-datos-limpios/CALLEJERO_VIGENTE_NUMERACIONES_202501.xlsx')

In [1212]:
#datos_pisosmadrid_normal = pd.read_excel("PisosMadrid.xlsx")
#datos_pisosmadrid_detalles = pd.read_excel("Detalle_pisosmadrid.xlsx")
#dbcp_ordenado= pd.read_excel("distr_barrios_codpostal.xlsx")
# redpiso_normal= pd.read_excel("Redpiso.xlsx")
# redpiso_detalles= pd.read_excel("Detalle_RedPiso.xlsx")

In [1213]:

def eliminar_acentos(nombre):
    return ''.join(c for c in unicodedata.normalize('NFD', nombre) if c.isascii())


In [1214]:
def extraer_calle(titulo):
    if isinstance(titulo, str):  # Comprobar que el valor es una cadena
        palabras_clave = ["CALLE", "AVENIDA", "PLAZA", "PASEO"]
        palabras = titulo.split(",")  # Dividir por comas
        for segmento in palabras:
            for palabra in palabras_clave:
                if palabra in segmento.upper():  # Insensible a mayúsculas/minúsculas
                    inicio = segmento.upper().find(palabra)  # Encontrar posición de la palabra clave
                    return segmento[inicio:].strip()  # Retornar desde la palabra clave hasta el final
    return np.nan  # Retornar el valor actual de 'calle' si no se encuentra la palabra clave


In [1215]:
def extraer_calle_1palabra(titulo):
    if isinstance(titulo, str):  # Comprobar que el valor es una cadena
        palabras_clave = ["CALLE", "AVENIDA", "PLAZA", "PASEO"]
        palabras = titulo.split()  # Dividir por espacios
        for i, palabra in enumerate(palabras):
            if palabra.upper() in palabras_clave:  # Buscar si la palabra es clave
                if i + 1 < len(palabras):  # Asegurarse de que hay una palabra siguiente
                    return palabras[i + 1]  # Retornar la palabra que sigue a la clave
    return None  # Retornar None si no se encuentra la calle o el valor no es válido


In [1216]:
def normalize_string(s):
    if pd.notna(s):
        s = unicodedata.normalize('NFD', s)  # Descomponer los caracteres con acento
        s = ''.join([c for c in s if not unicodedata.combining(c)])  # Eliminar los caracteres de acento
        s = s.lower().strip().replace(" ", "")  # Convertir a minúsculas
    return s

In [1217]:
#nuevas_columnas= ["barrio","cod_barrio","distrito","cod_distrito","codigo_Postal"]
#df_pisos[nuevas_columnas]=np.nan


zona_barrio_dict = {
    'Sanchinarro': {     
        'barrio': 'Valdefuentes'
    },
    'San Andrés': {       
        'barrio': 'Villaverde Alto'
    },
    'Virgen del Cortijo': {      
        'barrio': 'Valdefuentes'
    },
    'Montecarmelo': {       
        'barrio': 'Mirasierra'
    }
    ,
    'Villaverde Bajo': {       
        'barrio': 'Los Rosales'
    },
    'villaverdebajo': {       
        'barrio': 'Los Rosales'
    }
}




def clasificar_distrito_y_barrio_por_columnas(df, df_barrios,campo):
    # Recorrer las filas del dataframe
    for idx, fila in df.iterrows():
        # Recorrer las columnas en orden de prioridad: titulo, nombre_oficina
        for columna in [campo]:
            valor = fila[columna]
            
            # Verificamos que el valor no sea nulo
            if pd.notna(valor):
                valor_normalizado = normalize_string(valor)

                 # Reemplazar valores específicos en la ubicación normalizada
                if 'sanandres' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('sanandres', 'villaverdealto')
                elif 'salvador' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('salvador', 'elsalvador')
                elif 'san andres' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('san andres', 'villaverdealto')
                elif 'palos de moguer' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('palos de moguer', 'palosdemoguer')
                elif 'virgendelcortijo-manoteras' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('virgendelcortijo-manoteras', 'valdefuentes')
                elif 'sanchinarro' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('sanchinarro', 'valdefuentes')
                elif 'montecarmelo-elgoloso' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('montecarmelo-elgoloso', 'elgoloso')
                elif 'montecarmelo' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('montecarmelo', 'mirasierra')
                elif 'ambroz' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('ambroz', 'cascohistoricodevicalvaro')
                elif 'carmenes' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('carmenes', 'loscarmenes')
                elif 'paudecarabanchel' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('paudecarabanchel', 'buenavista')
                elif 'malasaña' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('malasaña', 'universidad')
                #elif 'centrohistorico' in valor_normalizado:
                    #valor_normalizado = valor_normalizado.replace('centrohistorico', 'cascohistoricodevicalvaro')
                elif 'lastablas' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('lastablas', 'valverde')
                elif 'malasana' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('malasana', 'universidad')
                elif 'lavapies' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('lavapies', 'embajadores')
                elif 'bernabeu' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('bernabeu', 'elviso')
                elif 'villaverdebajo' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('villaverdebajo', 'losrosales')
                elif 'orense' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('orense', 'cuatrocaminos')
                elif 'aguilas' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('aguilas', 'lasaguilas')
                elif 'lasmercedes' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('lasmercedes', 'rejas')
                elif 'granvia' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('granvia', 'justicia')
                elif 'chueca' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('chueca', 'justicia')
                elif 'puertadehierro' in valor_normalizado:
                    valor_normalizado = valor_normalizado.replace('puertadehierro', 'ciudaduniversitaria')
                
                # Verificamos si el valor normalizado contiene el nombre de un barrio
                barrio_encontrado = next((barrio for barrio in df_barrios["barrio_n"] if barrio in valor_normalizado), None)
                
                if barrio_encontrado and pd.isna(fila["barrio"]):
                    df.at[idx, "barrio"] = df_barrios.query(f"barrio_n == '{barrio_encontrado}'")["barrio"].values[0]  # Asignar el nombre del barrio a la columna "barrio"
                    # Buscar los valores cod_barrio, distrito y cod_distrito en df_barrios
                    result = df_barrios.query(f"barrio_n == '{barrio_encontrado}'")
                    if not result.empty:
                        cod_distrito = result["cod_distrito"].values[0]
                        cod_barrio = result["cod_barrio"].values[0]
                        codigo_Postal=result["codigo_Postal"].values[0]
                        distrito_result = result["distrito"].values[0]
                        df.at[idx, "cod_barrio"] = cod_barrio
                        df.at[idx, "cod_distrito"] = cod_distrito
                        df.at[idx, "distrito"] = distrito_result
                        df.at[idx, "codigo_Postal"] = codigo_Postal
                # Verificamos si el valor normalizado contiene el nombre de un distrito
                distrito_encontrado = next((distrito for distrito in df_barrios["distrito_n"] if distrito in valor_normalizado), None)
                if distrito_encontrado and pd.isna(fila["distrito"]):
                    df.at[idx, "distrito"] = df_barrios.query(f"distrito_n == '{distrito_encontrado}'")["distrito"].values[0]
                    result = df_barrios.query(f"distrito_n == '{distrito_encontrado}'")                    
                    cod_distrito = result["cod_distrito"].values[0]
                    df.at[idx, "cod_distrito"] = cod_distrito
            else:
                #zona_encontrado = next((barrio for barrio in df_barrios["barrio_n"] if barrio in valor_normalizado), None)
                barrio_encontrado = next((zona_barrio_dict[zona]["barrio"] for zona in zona_barrio_dict if zona_barrio_dict[zona]["barrio"] in df_barrios["barrio_n"]), None)
                if barrio_encontrado and pd.isna(fila["barrio"]):
                    df.at[idx, "barrio"] = df_barrios.query(f"barrio_n == '{barrio_encontrado}'")["barrio"].values[0]  # Asignar el nombre del barrio a la columna "barrio"
                    # Buscar los valores cod_barrio, distrito y cod_distrito en df_barrios
                    result = df_barrios.query(f"barrio_n == '{barrio_encontrado}'")
                    if not result.empty:
                        cod_distrito = result["cod_distrito"].values[0]
                        cod_barrio = result["cod_barrio"].values[0]
                        codigo_Postal=result["codigo_Postal"].values[0]
                        distrito_result = result["distrito"].values[0]
                        df.at[idx, "cod_barrio"] = cod_barrio
                        df.at[idx, "cod_distrito"] = cod_distrito
                        df.at[idx, "distrito"] = distrito_result
                        df.at[idx, "codigo_Postal"] = codigo_Postal

In [1218]:
datos_pisosmadrid_normal.head()

,Campo,Campo1,Precio,dormitorios,banos,m2construidos,Descripción,btn
0,https://www.pisosmadrid.com.es/propiedad/4086665-piso-en-venta-en-ibiza-madrid,"Piso en venta en Ibiza, Madrid",520.000 €,3 Dormitorio(s),1 baño,70 m2,"Precioso piso reformado de 60m2 enfrente del Retiro, súper luminoso, todas las estancias con ventana exterior.~~Piso de 3 habitaciones, 1 baño con plato de ducha, salón, cocina totalmente equipada, interior con muchísima...",Detalles
1,https://www.pisosmadrid.com.es/propiedad/4086544-piso-en-venta-en-madrid,Piso en venta en Madrid,329.000 €,3 Dormitorio(s),2 Baño(s),95 m2,"~SERVICHECK INMOBILIARIA comercializa un precioso pisode 3 dormitorios, dos baños, con trastero, plaza de garaje y jardín en excelente ubicación.~¡Oportunidad única en una de las mejores zonas residenciales de Madrid!~~-...",Detalles
2,https://www.pisosmadrid.com.es/propiedad/4086503-piso-en-venta-en-arganzuela-madrid,"Piso en venta en Arganzuela, Madrid",740.000 €,4 Dormitorio(s),3 Baño(s),158 m2,"Se ofrece una magnífica oportunidad para adquirir un amplio piso en el consolidado barrio de Delicias, en Madrid. La vivienda, con 158 metros cuadrados construidos, se encuentra en un edificio bien situado, ofreciendo un...",Detalles
3,https://www.pisosmadrid.com.es/propiedad/4086495-piso-en-venta-en-los-angeles-madrid,"Piso en venta en Los Angeles, Madrid",70.256 €,3 Dormitorio(s),1 baño,57 m2,"Vivienda situada en Calle Godella,, en Madrid capital. VIVIENDA CON INCIDENCIAS: INMUEBLE SIN POSESIÓN. ~~El estado de ocupado y/o los impedimentos para que el inmueble pueda ser visitado libremente por los técnicos impi...",Detalles
4,https://www.pisosmadrid.com.es/propiedad/4086493-piso-en-venta-en-madrid,Piso en venta en Madrid,97.920 €,2 Dormitorio(s),1 baño,50 m2,"Vivienda situada en Calle Transmonte,, en Madrid capital.. VIVIENDA CON INCIDENCIAS: INMUEBLE SIN POSESIÓN. ~~El estado de ocupado y/o los impedimentos para que el inmueble pueda ser visitado libremente por los técnicos ...",Detalles


In [1219]:
# Bien aqui vamos a construir una funcion que recorra todo el dataframe y tome los caracteres especiales 
# las comas, los puntos, entre otros y los reemplaza con nada o sea basicamente eliminarlos. 

#silvia esta funcion elimina sin discriminar, la comento

#def limpiar_texto(x):
 #   return re.sub(r'[^\w\s]','',unidecode(str(x)))

#datos_pisosmadrid_normal = datos_pisosmadrid_normal.apply(lambda col: col.apply(limpiar_texto)if col.name != "Campo"else col)

In [1220]:
#  Aqui creamos una funcion que recorrera la columna que especificamos quitanto el valor que nosotoros le digamos y sin distincion 
# alguna sobre si es mayuscula o minuscula.                                                                                 
#def eliminar_palabra(palabra):
 #   return re.sub(r'(?i)\bdormitorios\b', '', palabra)

#datos_pisosmadrid_normal["dormitorios"]= datos_pisosmadrid_normal["dormitorios"].apply(eliminar_palabra)
#def eliminar_palabra(palabra):
 #   return re.sub(r'(?i)\bdormitorio\b', '', palabra)

#datos_pisosmadrid_normal["dormitorios"]= datos_pisosmadrid_normal["dormitorios"].apply(eliminar_palabra)
#def eliminar_palabra(palabra):
 #   return re.sub(r'(?i)\bs\b', '', palabra)

#datos_pisosmadrid_normal["dormitorios"]= datos_pisosmadrid_normal["dormitorios"].apply(eliminar_palabra)

# --------------------------------------------------------------------------
# Usar regex para eliminar las palabras o caracteres específicos
datos_pisosmadrid_normal["dormitorios"] = datos_pisosmadrid_normal["dormitorios"].replace(to_replace=r"(Dormitorio|dormitorios|\(s\))",  value="",  regex=True)

datos_pisosmadrid_normal["banos"] = datos_pisosmadrid_normal["banos"].replace(to_replace=r"(baño|Baño|baños|\(s\))",  value="",  regex=True)

datos_pisosmadrid_normal["Precio"] = datos_pisosmadrid_normal["Precio"].replace(to_replace=r"(€)",  value="",  regex=True)

#def eliminar_palabra(palabra):
 #   return re.sub(r'(?i)\bbaño\b', '', palabra)

#datos_pisosmadrid_normal["banos"]= datos_pisosmadrid_normal["banos"].apply(eliminar_palabra)

# --------------------------------------------------------------------------
#def eliminar_palabra(palabra):
 #   return re.sub(r'(?i)\bbaños\b', '', palabra)

#datos_pisosmadrid_normal["banos"]= datos_pisosmadrid_normal["banos"].apply(eliminar_palabra)
# --------------------------------------------------------------------------

def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bm2\b', '', palabra)

datos_pisosmadrid_normal["m2construidos"]= datos_pisosmadrid_normal["m2construidos"].apply(eliminar_palabra)
# --------------------------------------------------------------------------

#def eliminar_palabra(palabra):
 #   return re.sub(r'(?i)\b€\b', '', palabra)

#datos_pisosmadrid_normal["Precio"]= datos_pisosmadrid_normal["Precio"].apply(eliminar_palabra)


In [1221]:
# silvia añado el tipo_lugar y elimino los que no se van a analizar y campo referencia para el merge

In [1222]:
# Función para extraer números entre '/' y '-'
def extraer_numeros(texto):
    patron = r"/(\d+)-"
    resultado = re.search(patron, texto)
    return resultado.group(1) if resultado else None

# Aplicar la función a la columna 'campo'
datos_pisosmadrid_normal["referencia"]=  datos_pisosmadrid_normal['Campo'].apply(extraer_numeros)


In [1223]:

# Extraer la primera palabra de la columna 'titulo' 
datos_pisosmadrid_normal['tipo_lugar'] = datos_pisosmadrid_normal['Campo1'].apply(lambda x: x.split()[0]) 
#silvia añado la elimninacion de aquello que no son pisos
# Lista de tipos a eliminar
tipos_eliminar = ['Local', 'Garaje', 'Parking', 'Oficina', 'Terreno', 'Nave', 'Edificio', 'Solar', 'Parcela', 'Trastero']

# Filtrar el DataFrame para eliminar las filas con los valores especificados
datos_pisosmadrid_normal= datos_pisosmadrid_normal[~datos_pisosmadrid_normal['tipo_lugar'].isin(tipos_eliminar)]



In [1224]:
# Aqiu se asignaron los nuesvos nombres por cada columna de forma indivual.
# El proceso tambien se puede hacer de forma mas corta.... Declaras una variable nueva con los nombres que quieres darles a las columnas
# en el orden que quieres poner los nombres y despues los asignas a las columnas.
#datos_pisosmadrid_normal = datos_pisosmadrid_normal.rename(columns={"Campo":"url_webside"})
datos_pisosmadrid_normal = datos_pisosmadrid_normal.rename(columns={"Campo":"url"})
 # --------------------------------------------------------------------------                  
#datos_pisosmadrid_normal = datos_pisosmadrid_normal.rename(columns={"Campo1":"descripcion"})
datos_pisosmadrid_normal = datos_pisosmadrid_normal.rename(columns={"Campo1":"title"})
 # --------------------------------------------------------------------------
                    
datos_pisosmadrid_normal = datos_pisosmadrid_normal.rename(columns={"Precio":"precio"})
  # --------------------------------------------------------------------------
                   
#datos_pisosmadrid_normal = datos_pisosmadrid_normal.rename(columns={"dormitorios":"cant_dormitorios"})
 # --------------------------------------------------------------------------
                    
#datos_pisosmadrid_normal = datos_pisosmadrid_normal.rename(columns={"banos":"cant_banos"})
# --------------------------------------------------------------------------

#datos_pisosmadrid_normal = datos_pisosmadrid_normal.rename(columns={"m2construidos":"metros_cuadrados_construidos"})

datos_pisosmadrid_normal = datos_pisosmadrid_normal.rename(columns={"m2construidos":"m2_constr"})
  # --------------------------------------------------------------------------
  # --------------------------------------------------------------------------
                   
#datos_pisosmadrid_normal = datos_pisosmadrid_normal.rename(columns={"Descripción":"descripcion_extendida"})
datos_pisosmadrid_normal = datos_pisosmadrid_normal.rename(columns={"Descripción":"descripcion_larga"})
datos_pisosmadrid_normal = datos_pisosmadrid_normal.rename(columns={"tipo_lugar":"tipologia"})
   # --------------------------------------------------------------------------
                  
datos_pisosmadrid_normal = datos_pisosmadrid_normal.drop(columns=["btn"])  

In [1225]:
def crear_columnas(df,nueva_col,valor_busqueda):
   # df[nueva_col]= np.nan
    for index, row in df.iterrows():
     for col in df.columns[:-1]:
        if pd.notna(row[col]):
            for valor in valor_busqueda:
                if valor in str(row[col]):
                    df.at[index, nueva_col] =valor 
                    #df.at[index, col]= np.nan
                    break


In [1226]:
valores_buscar = [
    "Planta Sótano","planta sotano","Sótano","sotano",
    "planta baja","Planta Baja","bajo","Bajo",
    "primera planta", "Primera planta", "1ª planta", "1a planta",
    "segunda planta", "Segunda planta", "2ª planta", "2a planta",
    "tercera planta", "Tercera planta", "3ª planta", "3a planta",
    "cuarta planta", "Cuarta planta", "4ª planta", "4a planta",
    "quinta planta", "Quinta planta", "5ª planta", "5a planta",
    "sexta planta", "Sexta planta", "6ª planta", "6a planta",
    "séptima planta", "Séptima planta", "7ª planta", "7a planta",
    "octava planta", "Octava planta", "8ª planta", "8a planta",
    "novena planta", "Novena planta", "9ª planta", "9a planta",
    "décima planta", "Décima planta", "10ª planta", "10a planta",
    "undécima planta", "Undécima planta", "11ª planta", "11a planta",
    "duodécima planta", "Duodécima planta", "12ª planta", "12a planta",
    "decimotercera planta", "Decimotercera planta", "13ª planta", "13a planta",
    "decimocuarta planta", "Decimocuarta planta", "14ª planta", "14a planta",
    "decimoquinta planta", "Decimoquinta planta", "15ª planta", "15a planta",
    "decimosexta planta", "Decimosexta planta", "16ª planta", "16a planta",
    "decimoséptima planta", "Decimoséptima planta", "17ª planta", "17a planta",
    "decimoctava planta", "Decimoctava planta", "18ª planta", "18a planta",
    "decimonovena planta", "Decimonovena planta", "19ª planta", "19a planta",
    "vigésima planta", "Vigésima planta", "20ª planta", "20a planta"
]



crear_columnas(datos_pisosmadrid_normal, "planta", valores_buscar)

In [1227]:
datos_pisosmadrid_normal["planta"].value_counts(dropna=False)

planta
NaN               750
planta baja        27
primera planta     18
segunda planta     16
tercera planta     16
cuarta planta      16
bajo               14
quinta planta       6
4ª planta           4
séptima planta      4
2ª planta           3
sexta planta        3
Primera planta      2
3ª planta           2
5ª planta           2
Bajo                2
8ª planta           1
novena planta       1
1ª planta           1
Cuarta planta       1
1a planta           1
8a planta           1
7ª planta           1
Sexta planta        1
Name: count, dtype: int64

In [1228]:
datos_pisosmadrid_normal.columns

Index(['url', 'title', 'precio', 'dormitorios', 'banos', 'm2_constr',
       'descripcion_larga', 'referencia', 'tipologia', 'planta'],
      dtype='object')

### **_Aqui se termina la parte de --pisosmadrid_normal-- y empieza la de --pisosmadrid_detalles--_**

In [1229]:
datos_pisosmadrid_detalles.head()

,Título,Campo1,Campo2,Campo3,Campo4,Campo5,Campo6,Campo7,Tipo,Ciudad,Campo8,Campo9,Area,Campo10,Provincia,Campo11,Campo12,Campo13,Campo14,Campo15,Metros,Campo16,Piscina,Campo17,Garaje,Jardín,Campo18,Campo19,Referencia,Publicado,Visitas,Texto
0,"Piso en venta en Ibiza, Madrid",3 Dormitorio(s),1 baño,70 m2,Descripción:,"Descripción:\n Precioso piso reformado de 60m2 enfrente del Retiro, súper luminoso, todas las estancias con ventana exterior.~~Piso de 3 habitaciones, 1 baño con plato de ducha, salón, cocina totalmente equipada, interior con muchísima luz.~~Techos altos, suelos de tarima, ventanas blancas de climalit oscilobatientes, puertas lacadas en blanco, aire acondicionado en el salón.~~Ascensor, calefacción central, agua caliente por caldera de gas natural.~~Ubicación excelente en frente del Parque del Retiro.~~No dudes en venir a visitarlo!",Datos principales:,Tipo:,Piso,Madrid,Ciudad:,Area:,Ibiza,Provincia:,Madrid,Dormitorio(s):,3.0,Baño(s):,1.0,Metros:,70.0,Piscina:,No,Garaje:,No,No,Jardín:,Características:\n\t\t\t\tCalefacción,PM4086665,24/10/2024,4.0,520.000 €
1,Piso en venta en Madrid,3 Dormitorio(s),2 Baño(s),95 m2,Descripción:,"Descripción:\n\t\t\t\t~SERVICHECK INMOBILIARIA comercializa un precioso pisode 3 dormitorios, dos baños, con trastero, plaza de garaje y jardín en excelente ubicación.~¡Oportunidad única en una de las mejores zonas residenciales de Madrid!~~-Para una mejor experiencia visual entra a la visita en 3D"".~-(Para mejor comunicación enviar un WhatsApp) Para visitas presenciales, preguntar por Emiliano Correa al 602_250_283 (Para mejor comunicación enviar un WhatsApp)~~~El piso cuenta con una distribución cuidadosamente diseñada para brindarte la máxima comodidad y funcionalidad.Está en una tercera planta con ascensor, construido en 2007 y en muy buen estado de conservación.~~Dispone de 157 m² construidos en total: ~VIVIENDA 95\tm2\t~APARCAMIENTO 10 m2\t~ALMACEN 5\tm2\t~ELEMENTOS COMUNES 47 m2~~Distribuidos de manera funcional para aprovechar al máximo cada espacio.~~-Las características:~~-Hall de entrada.~-Salón comedor amplio y muy luminoso con vistas al interior de la finca, ideal para relajarse o disfrutar en familia.~-Cocina totalmente equipada con acceso a una terraza tendedero.~-Tres dormitorios con armarios empotrados, perfectos para mantener todo en orden.~-Dos cuartos de baño, uno con plato de ducha y otro en suite con bañera.~-Armario empotrado adicional en el pasillo, para mayor espacio de almacenamiento.~~La vivienda cuenta con calefacción individual por gas natural, garantizando confort en los meses de invierno y ""la preinstalación del aire acondicionado"". Además, se incluye una plaza de garaje y un trastero en el precio, brindando soluciones prácticas para almacenamiento adicional.~~El edificio cuenta con una amplia zona ajardinada, perfecta para pasear o disfrutar del aire libre sin salir de casa. ~Además, su ubicación es excelente, frente al Centro Deportivo Municipal Juan de Dios Román y a solo 7 minutos caminando del metro Las Suertes, con fácil acceso a la M45 y la A3. Cerca de todo tipo de servicios: centro de salud, colegios, guarderías, y el Centro Comercial La Gavia.~~-La ubicación: Calle de José Gutiérrez Maroto, Ensanche de VallecasBarrio Ensanche de Vallecas - La Gavia.Madrid.~~Una oportunidad ideal tanto para vivienda habitual como para inversión en una zona residencial tranquila y bien comunicada.",Datos principales:,Tipo:,Piso,Madrid,Ciudad:,Area:,NaN,Provincia:,Madrid,Dormitorio(s):,3.0,Baño(s):,2.0,Metros:,95.0,Piscina:,No,Garaje:,Sí,Sí,Jardín:,Características:\n\t\t\t\tAmueblado - Calefacción - Trastero - Videoportero,PM4086544,24/10/2024,2.0,329.000 €
2,"Piso en venta en Arganzuela, Madrid",4 Dormitorio(s),3 Baño(s),158 m2,Descripción:,"Descripción:\n\t\t\t\tSe ofrece una magnífica oportunidad para adquirir un amplio piso en el consolidado barrio de Delicias, en Madrid. La vivienda, con 158 metros cuadrados construidos, se encuentra en un edificio bien situado,

In [1230]:
datos_pisosmadrid_detalles["Campo18"].unique()

array(['Jardín:', nan], dtype=object)

In [1231]:
datos_pisosmadrid_detalles["Campo12"].value_counts()

Campo12
0.0     327
3.0     319
2.0     272
1.0     109
4.0      98
5.0      34
6.0      12
7.0       7
8.0       5
9.0       2
10.0      1
20.0      1
Name: count, dtype: int64

In [1232]:
datos_pisosmadrid_detalles["Campo1"].value_counts(dropna=False)

Campo1
0 Dormitorio(s)     327
3 Dormitorio(s)     319
2 Dormitorio(s)     272
1 dormitorio        109
4 Dormitorio(s)      98
5 Dormitorio(s)      34
6 Dormitorio(s)      12
7 Dormitorio(s)       7
8 Dormitorio(s)       5
9 Dormitorio(s)       2
NaN                   1
10 Dormitorio(s)      1
20 Dormitorio(s)      1
Name: count, dtype: int64

In [1233]:
datos_pisosmadrid_detalles.shape

(1188, 32)

In [1234]:
datos_pisosmadrid_detalles.query("Campo1.isna()")

,Título,Campo1,Campo2,Campo3,Campo4,Campo5,Campo6,Campo7,Tipo,Ciudad,Campo8,Campo9,Area,Campo10,Provincia,Campo11,Campo12,Campo13,Campo14,Campo15,Metros,Campo16,Piscina,Campo17,Garaje,Jardín,Campo18,Campo19,Referencia,Publicado,Visitas,Texto
127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1235]:
datos_pisosmadrid_detalles = datos_pisosmadrid_detalles[datos_pisosmadrid_detalles["Título"].notnull()]


In [1236]:
datos_pisosmadrid_detalles["Campo14"].value_counts()

Campo14
1.0     416
2.0     350
0.0     266
3.0     108
4.0      25
5.0       9
6.0       6
7.0       3
31.0      1
10.0      1
11.0      1
8.0       1
Name: count, dtype: int64

In [1237]:
datos_pisosmadrid_detalles["Campo2"].value_counts()

Campo2
1 baño        416
2 Baño(s)     350
0 Baño(s)     266
3 Baño(s)     108
4 Baño(s)      25
5 Baño(s)       9
6 Baño(s)       6
7 Baño(s)       3
31 Baño(s)      1
10 Baño(s)      1
11 Baño(s)      1
8 Baño(s)       1
Name: count, dtype: int64

In [1238]:
datos_pisosmadrid_detalles["Campo3"].value_counts()

Campo3
70 m2      21
0 m2       18
22 m2      18
m2         16
80 m2      15
           ..
18 m2       1
314 m2      1
15 m2       1
208 m2      1
3995 m2     1
Name: count, Length: 305, dtype: int64

In [1239]:
datos_pisosmadrid_detalles["Metros"].value_counts()

Metros
70.0      21
0.0       18
22.0      18
80.0      15
65.0      15
          ..
726.0      1
769.0      1
460.0      1
1600.0     1
3995.0     1
Name: count, Length: 304, dtype: int64

In [1240]:
datos_pisosmadrid_normal.query("url=='https://www.pisosmadrid.com.es/propiedad/4086303-piso-en-venta-en-malasana-madrid'")

,url,title,precio,dormitorios,banos,m2_constr,descripcion_larga,referencia,tipologia,planta
7,https://www.pisosmadrid.com.es/propiedad/4086303-piso-en-venta-en-malasana-madrid,"Piso en venta en Malasaña, Madrid",1.400.000,5,2,212,"¡Gran Oportunidad en Malasaña! Vivienda amplia y luminosa para reformar ~~InmoRenta te presenta esta magnífica propiedad para reformar ubicada en la calle Corredera Baja de San Pablo, en pleno corazón de Malasaña, uno de...",4086303,Piso,NaN


In [1241]:
# Aqui se eliminaron una lista de columnas elegidas por mi. Se hizo un previo analisis del valor que proporcionaban cada una y la conclusion 
# fue que el valor que proprcionaban era igual o menor que nada.
datos_pisosmadrid_detalles = datos_pisosmadrid_detalles.drop(columns=["Campo4","Campo6","Campo7","Campo8","Campo9","Campo10","Campo15","Campo16","Campo17","Campo18","Provincia","Campo11",
                            "Campo12","Campo14", "Campo13","Metros"])

In [1242]:
#quito la limpieza masiva de columnas

#def limpiar_texto(x):
 #   return re.sub(r'[^\w\s]','',unidecode(str(x)))
#datos_pisosmadrid_detalles = datos_pisosmadrid_detalles.apply(lambda col: col.apply(limpiar_texto)if col.name !="Publicado" else col)

In [1243]:
datos_pisosmadrid_detalles.columns

Index(['Título', 'Campo1', 'Campo2', 'Campo3', 'Campo5', 'Tipo', 'Ciudad',
       'Area', 'Piscina', 'Garaje', 'Jardín', 'Campo19', 'Referencia',
       'Publicado', 'Visitas', 'Texto'],
      dtype='object')

In [1244]:

#nuevos_nombres = ["descripcion","cant_dormitorios","cant_banos","m_cuadrados","descripcion2","tipo_lugar","ciudad","barrio/localidad/distrito","piscina","garaje",
          #        "jardin","descripcion3","referencia","fecha_publicacion","n.visitas","precio"]
#datos_pisosmadrid_detalles.columns= nuevos_nombres

In [1245]:
nuevos_nombres = {
    'Título': 'title',
    'Campo1': 'dormitorios',
    'Campo2': 'banos',
    'Campo3': 'm2_constr',
    'Campo5': 'descripcion_larga',
    'Tipo': 'tipologia',
    'area': 'zona',
    'Campo19': 'caracteristicas',
    'Publicado': 'fecha_publicacion',
    'Visitas': 'num_visitas',
    'Texto': 'precio',
}

# Renombrar las columnas
datos_pisosmadrid_detalles = datos_pisosmadrid_detalles.rename(columns=nuevos_nombres)
# Convertir los nombres de las columnas a minúsculas
# Función para eliminar acentos


# Aplicar la función a los nombres de las columnas
datos_pisosmadrid_detalles.columns = [eliminar_acentos(col) for col in datos_pisosmadrid_detalles.columns]
datos_pisosmadrid_detalles.columns = datos_pisosmadrid_detalles.columns.str.lower()

In [1246]:
# Aqui creamos una funcion que recorrera la columna que especificamos quitanto el valor que nosotoros le digamos y sin distincion 
# alguna sobre si es mayuscula o minuscula.

# --------------------------------------------------------------------------

def eliminar_palabra(palabra):
    return re.sub(r'(?i)\bm2\b', '', palabra)

datos_pisosmadrid_detalles["m2_constr"]= datos_pisosmadrid_detalles["m2_constr"].apply(eliminar_palabra)


datos_pisosmadrid_detalles["dormitorios"] = datos_pisosmadrid_detalles["dormitorios"].replace(to_replace=r"(dormitorio|Dormitorio|dormitorios|\(s\))",  value="",  regex=True)

datos_pisosmadrid_detalles["banos"] = datos_pisosmadrid_detalles["banos"].replace(to_replace=r"(baño|Baño|baños|\(s\))",  value="",  regex=True)

datos_pisosmadrid_detalles["precio"] = datos_pisosmadrid_detalles["precio"].replace(to_replace=r"(€)",  value="",  regex=True)

# silvia añado la elimninacion de aquello que no son pisos

In [1247]:
datos_pisosmadrid_detalles['tipologia'].value_counts(dropna=False)

tipologia
Piso                       707
Local                      112
Garaje                      69
Parking                     65
Apartamento                 45
Atico                       43
Oficina                     17
Apartamento planta baja     17
Duplex                      16
Terreno                     15
Estudio                     15
Nave                        14
Adosado                     12
Chalet                       9
Edificio                     7
Loft                         6
Negocio                      5
Solar                        3
Pareado                      3
Parcela                      2
Casa                         2
Trastero                     1
Bungalow                     1
Villa                        1
Name: count, dtype: int64

In [1248]:
#silvia añado la elimninacion de aquello que no son pisos
# Lista de tipos a eliminar
tipos_eliminar = ['Local', 'Garaje', 'Parking', 'Oficina', 'Terreno', 'Nave', 'Edificio', 'Negocio','Solar', 'Parcela', 'Trastero']

# Filtrar el DataFrame para eliminar las filas con los valores especificados
datos_pisosmadrid_detalles= datos_pisosmadrid_detalles[~datos_pisosmadrid_detalles['tipologia'].isin(tipos_eliminar)]
#elimino PM de referencia
datos_pisosmadrid_detalles["referencia"] = datos_pisosmadrid_detalles["referencia"].str.replace("PM", "")

In [1249]:
datos_pisosmadrid_detalles["referencia"].value_counts(dropna=False)

referencia
4086665    1
3974110    1
3978041    1
3977655    1
3975253    1
          ..
4049210    1
4048885    1
4047995    1
4047935    1
3863841    1
Name: count, Length: 877, dtype: int64

In [1250]:
# Aqui vamos a convertir el tipo de dato que tiene una columna en especifico al tipo datetime. 
datos_pisosmadrid_detalles["fecha_publicacion"]=pd.to_datetime(datos_pisosmadrid_detalles["fecha_publicacion"], dayfirst=True)

In [1251]:
datos_pisosmadrid_detalles["caracteristicas"].value_counts(dropna=False)

caracteristicas
Características:                                                                                             247
Características:\n\t\t\t\tCalefacción                                                                        174
Características:\n\t\t\t\tCalefacción - Videoportero                                                          97
Características:\n\t\t\t\tCalefacción - Trastero - Videoportero                                               66
Características:\n\t\t\t\tCalefacción - Trastero                                                              50
Características:\n\t\t\t\tAmueblado - Calefacción - Videoportero                                              50
Características:\n\t\t\t\tAmueblado - Calefacción                                                             49
Características:\n\t\t\t\tAmueblado - Calefacción - Trastero - Videoportero                                   17
Características:\n\t\t\t\tVideoportero                                          

In [1252]:

#datos_pisosmadrid_detalles["caracteristicas"]= datos_pisosmadrid_detalles["caracteristicas"].str.replace("'Caracteristicas:'", "",regex=True)

In [1253]:
# Aqui vamos a hacer una serie de reemplazamientos indivuales . 


#datos_pisosmadrid_detalles["caracteristicas"]= datos_pisosmadrid_detalles["caracteristicas"].str.replace("'Caracteristicas\n\t\t\t\t'", "",regex=True)
#datos_pisosmadrid_detalles["caracteristicas"]= datos_pisosmadrid_detalles["caracteristicas"].str.replace(r'Caracteristicas',"",regex=True).replace("",np.nan)

#datos_pisosmadrid_detalles["descripcion2"]= datos_pisosmadrid_detalles["descripcion2"].str.replace(r'Descripcion\n\t\t\t\t', "",regex=True)
#datos_pisosmadrid_detalles["descripcion2"]= datos_pisosmadrid_detalles["descripcion2"].str.replace(r'Descripcion\n', "",regex=True)


In [1254]:
columnas_01= ["piscina","garaje","jardin"]

datos_pisosmadrid_detalles[columnas_01]=datos_pisosmadrid_detalles[columnas_01].replace({"Sí":1,"No":0})

C:\Users\extas\AppData\Local\Temp\ipykernel_12100\141113248.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  datos_pisosmadrid_detalles[columnas_01]=datos_pisosmadrid_detalles[columnas_01].replace({"Sí":1,"No":0})


In [1255]:
# Aqui vamos a crear una variabel que guarde un diccionario con las palabras que quremos tomar de la columna caracteristicas y el nombre
# de la columna a la que queremos que la palabra se vaya.
palabras_a_columnas = {
    "Calefacción" : "calefaccion",
    "Trastero"    : "trastero",
    "Amueblado"   :"amueblado",
    "Chimenea"    :"chimenea",
    #"Lavadero"    :"lavadero",
    "Videoportero":"videoportero",
    "Alarma"      :"alarma",
    "Aire acondicionado": "aire_acondicionado"
}

# Aqui creamos una variable donde guardaremos el nombre de la columna de donde se van a extraer todas esas palabras mencionadas antes.
columna_original = 'caracteristicas'

# Aqui vamos a crear un bucle for que itera sobre las columnas y encuentra los valores que escribimos antes. 
for palabra, nueva_columna in palabras_a_columnas.items():
    # Usar apply para crear la columna con 1 si la palabra está presente, de lo contrario 0
    datos_pisosmadrid_detalles[nueva_columna] = datos_pisosmadrid_detalles[columna_original].apply(lambda x: 1 if isinstance(x, str) and palabra.lower() in x.lower() else np.nan)

# Este bucle itera sobre las claves del diccionario palabras_a_columnas. 

#for palabra in palabras_a_columnas.keys():
 #   datos_pisosmadrid_detalles[columna_original] = datos_pisosmadrid_detalles[columna_original].str.replace(rf'\b{palabra}\b', '', regex=True)
#datos_pisosmadrid_detalles[columna_original] = datos_pisosmadrid_detalles[columna_original].str.strip()
# Reemplazar valores vacíos o "nan" con NaN en la columna original
#datos_pisosmadrid_detalles[columna_original] = datos_pisosmadrid_detalles[columna_original].replace(["", "nan"], np.nan)



In [1256]:
datos_pisosmadrid_detalles["aire_acondicionado"].value_counts(dropna=False)

aire_acondicionado
NaN    840
1.0     37
Name: count, dtype: int64

In [1257]:
# Aqui eliminamos la columna de la que sacamos todas las palabras anteriores, ahora solo queda con un nulos asi que hay que dejarla ir.
datos_pisosmadrid_detalles= datos_pisosmadrid_detalles.drop("caracteristicas",axis=1)

In [1258]:
# datos_pisosmadrid_detalles = datos_pisosmadrid_detalles.to_csv("pisosmadrid_detalles_tratado.csv")

### **_EDA/ Exploratory Data Analysis y Preprocesamiento PisosMadrid normal y detalle_**

In [1259]:
datos_pisosmadrid_detalles.shape

(877, 22)

In [1260]:
datos_pisosmadrid_normal.shape

(893, 10)

In [1261]:
datos_pisosmadrid_normal.head(3)

,url,title,precio,dormitorios,banos,m2_constr,descripcion_larga,referencia,tipologia,planta
0,https://www.pisosmadrid.com.es/propiedad/4086665-piso-en-venta-en-ibiza-madrid,"Piso en venta en Ibiza, Madrid",520.000,3,1,70,"Precioso piso reformado de 60m2 enfrente del Retiro, súper luminoso, todas las estancias con ventana exterior.~~Piso de 3 habitaciones, 1 baño con plato de ducha, salón, cocina totalmente equipada, interior con muchísima...",4086665,Piso,NaN
1,https://www.pisosmadrid.com.es/propiedad/4086544-piso-en-venta-en-madrid,Piso en venta en Madrid,329.000,3,2,95,"~SERVICHECK INMOBILIARIA comercializa un precioso pisode 3 dormitorios, dos baños, con trastero, plaza de garaje y jardín en excelente ubicación.~¡Oportunidad única en una de las mejores zonas residenciales de Madrid!~~-...",4086544,Piso,NaN
2,https://www.pisosmadrid.com.es/propiedad/4086503-piso-en-venta-en-arganzuela-madrid,"Piso en venta en Arganzuela, Madrid",740.000,4,3,158,"Se ofrece una magnífica oportunidad para adquirir un amplio piso en el consolidado barrio de Delicias, en Madrid. La vivienda, con 158 metros cuadrados construidos, se encuentra en un edificio bien situado, ofreciendo un...",4086503,Piso,NaN


In [1262]:
datos_pisosmadrid_detalles.head(3)

,title,dormitorios,banos,m2_constr,descripcion_larga,tipologia,ciudad,area,piscina,garaje,jardin,referencia,fecha_publicacion,num_visitas,precio,calefaccion,trastero,amueblado,chimenea,videoportero,alarma,aire_acondicionado
0,"Piso en venta en Ibiza, Madrid",3,1,70,"Descripción:\n Precioso piso reformado de 60m2 enfrente del Retiro, súper luminoso, todas las estancias con ventana exterior.~~Piso de 3 habitaciones, 1 baño con plato de ducha, salón, cocina totalmente equipada, interior con muchísima luz.~~Techos altos, suelos de tarima, ventanas blancas de climalit oscilobatientes, puertas lacadas en blanco, aire acondicionado en el salón.~~Ascensor, calefacción central, agua caliente por caldera de gas natural.~~Ubicación excelente en frente del Parque del Retiro.~~No dudes en venir a visitarlo!",Piso,Madrid,Ibiza,0,0,0,4086665,2024-10-24,4.0,520.000,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Piso en venta en Madrid,3,2,95,"Descripción:\n\t\t\t\t~SERVICHECK INMOBILIARIA comercializa un precioso pisode 3 dormitorios, dos baños, con trastero, plaza de garaje y jardín en excelente ubicación.~¡Oportunidad única en una de las mejores zonas residenciales de Madrid!~~-Para una mejor experiencia visual entra a la visita en 3D"".~-(Para mejor comunicación enviar un WhatsApp) Para visitas presenciales, preguntar por Emiliano Correa al 602_250_283 (Para mejor comunicación enviar un WhatsApp)~~~El piso cuenta con una distribución cuidadosamente diseñada para brindarte la máxima comodidad y funcionalidad.Está en una tercera planta con ascensor, construido en 2007 y en muy buen estado de conservación.~~Dispone de 157 m² construidos en total: ~VIVIENDA 95\tm2\t~APARCAMIENTO 10 m2\t~ALMACEN 5\tm2\t~ELEMENTOS COMUNES 47 m2~~Distribuidos de manera funcional para aprovechar al máximo cada espacio.~~-Las características:~~-Hall de entrada.~-Salón comedor amplio y muy luminoso con vistas al interior de la finca, ideal para relajarse o disfrutar en familia.~-Cocina totalmente equipada con acceso a una terraza tendedero.~-Tres dormitorios con armarios empotrados, perfectos para mantener todo en orden.~-Dos cuartos de baño, uno con plato de ducha y otro en suite con bañera.~-Armario empotrado adicional en el pasillo, para mayor espacio de almacenamiento.~~La vivienda cuenta con calefacción individual por gas natural, garantizando confort en los meses de invierno y ""la preinstalación del aire acondicionado"". Además, se incluye una plaza de garaje y un trastero en el precio, brindando soluciones prácticas para almacenamiento adicional.~~El edificio cuenta con una amplia zona ajardinada, perfecta para pasear o disfrutar del aire libre sin salir de casa. ~Además, su ubicación es excelente, frente al Centro Deportivo Municipal Juan de Dios Román y a solo 7 minutos caminando del metro Las Suertes, con fácil acceso a la M45 y la A3. Cerca de todo tipo de servicios: centro de salud, colegios, guarderías, y el Centro Comercial La Gavia.~~-La ubicación: Calle de José Gutiérrez Maroto, Ensanche de VallecasBarrio Ensanche de Vallecas - La Gavia.Madrid.~~Una oportunidad ideal tanto para vivienda habitual como para inversión en una zona residencial tranquila y bien comunicada.",Piso,Madrid,NaN,0,1,1,4086544,2024-10-24,2.0,329.000,1.0,1.0,1.0,NaN,1.0,NaN,NaN
2,"Piso en venta en Arganzuela, Madrid",4,3,158,"Descripción:\n\t\t\t\tSe ofrece una magnífica oportunidad para adquirir un amplio piso en el consolidado barrio de Delicias, en Madrid. La vivienda, con 158 metros cuadrados construidos, se encuentra en un edificio bien situado, ofreciendo una excelente oportunidad para aquellos que desean diseñar y personalizar su hogar, ya que el piso requiere una reforma integral y presenta numerosas posibilidades de redistribución y aprovechamiento de espacios.~~La distribución actual cuenta con una cocina espaciosa que dispone de entrada independiente de servicio, lo que aporta comodidad y funcionalidad a la vida cotidiana. El piso dispone de cuatro habitaciones amplias, una de ellas de

In [1263]:
#datos_pisosmadrid_detalles= datos_pisosmadrid_detalles.rename(columns={"m_cuadrados":"metros_cuadrados_construidos"})
#datos_pisosmadrid_detalles= datos_pisosmadrid_detalles.rename(columns={"descripcion2":"descripcion_extendida"})

In [1264]:
datos_pisosmadrid_detalles["fecha_publicacion"]=pd.to_datetime(datos_pisosmadrid_detalles["fecha_publicacion"],format='%Y-%m-%d',errors="coerce")

In [1265]:
# Silvia , esto no se puede asignar directamente

#datos_pisosmadrid_detalles["url_website"] = datos_pisosmadrid_normal["url_website"]

col_conversion= ["piscina","garaje","jardin","n.visitas","cant_dormitorios",
                 "precio","cant_banos","metros_cuadrados_construidos" ]
# Reemplazar 'nan' y cadenas vacías '' por 0 en todas las columnas relevantes
datos_pisosmadrid_detalles[col_conversion] = datos_pisosmadrid_detalles[col_conversion].replace({'nan': '0', '': '0'})

# Convertir las columnas a tipo numérico (float primero para manejar correctamente los valores)
datos_pisosmadrid_detalles[col_conversion] = datos_pisosmadrid_detalles[col_conversion].apply(pd.to_numeric, errors='coerce')

# Convertir las columnas a enteros, llenando NaN con 0 antes de la conversión
datos_pisosmadrid_detalles[col_conversion] = datos_pisosmadrid_detalles[col_conversion].fillna(0).astype("int64")


In [1266]:
datos_pisosmadrid_detalles.shape

(877, 22)

In [1267]:
datos_pisosmadrid_detalles.dtypes

title                         object
dormitorios                   object
banos                         object
m2_constr                     object
descripcion_larga             object
tipologia                     object
ciudad                        object
area                          object
piscina                        int64
garaje                         int64
jardin                         int64
referencia                    object
fecha_publicacion     datetime64[ns]
num_visitas                  float64
precio                        object
calefaccion                  float64
trastero                     float64
amueblado                    float64
chimenea                     float64
videoportero                 float64
alarma                       float64
aire_acondicionado           float64
dtype: object

In [1268]:
datos_pisosmadrid_detalles.isna().sum()

title                   0
dormitorios             0
banos                   0
m2_constr               0
descripcion_larga       0
tipologia               0
ciudad                  0
area                  119
piscina                 0
garaje                  0
jardin                  0
referencia              0
fecha_publicacion       0
num_visitas             0
precio                  0
calefaccion           302
trastero              699
amueblado             718
chimenea              873
videoportero          595
alarma                831
aire_acondicionado    840
dtype: int64

In [1269]:
datos_pisosmadrid_detalles["piscina"].unique()

array([0, 1], dtype=int64)

In [1270]:
datos_pisosmadrid_detalles["garaje"].unique()

array([0, 1], dtype=int64)

In [1271]:
datos_pisosmadrid_detalles["jardin"].unique()

array([0, 1], dtype=int64)

In [1272]:
datos_pisosmadrid_detalles["num_visitas"].unique()

array([ 4.,  2.,  3.,  6.,  5.,  1.,  7.,  8., 17., 13.,  9., 11., 10.,
       12., 19., 15., 16., 14., 18., 20.])

In [1273]:
datos_pisosmadrid_detalles["precio"].unique()

array(['520.000 ', '329.000 ', '740.000 ', '70.256 ', '97.920 ',
       '69.313 ', '1.400.000 ', '1.800.000 ', '380.000 ', '235.000 ',
       '1.354.900 ', '1.129.000 ', '320.000 ', '2.099.000 ', '960.000 ',
       '315.000 ', '220.000 ', '299.000 ', '2.350.000 ', '515.000 ',
       '870.000 ', '625.000 ', '250.000 ', '298.500 ', '322.900 ',
       '317.000 ', '322.000 ', '1.500.000 ', '189.900 ', '575.000 ',
       '675.000 ', '275.000 ', '370.000 ', '285.000 ', '475.000 ',
       '570.000 ', '1.300.000 ', '249.000 ', '460.000 ', '1.355.000 ',
       '54.000 ', '539.000 ', '600.000 ', '135.000 ', '843.800 ',
       '970.000 ', '850.000 ', '1.770.000 ', '655.000 ', '301.000 ',
       '1.150.000 ', '800.000 ', '239.000 ', '238.000 ', '210.000 ',
       '335.000 ', '1.550.000 ', '1.450.000 ', '500.000 ', '240.000 ',
       '2.475.000 ', '925.000 ', '1.140.000 ', '1.330.000 ', '650.000 ',
       '190.000 ', '173.000 ', '266.500 ', '230.000 ', '286.700 ',
       '459.000 ', '179.990 ', '40

In [1274]:
#columnas_texto = datos_pisosmadrid_detalles.select_dtypes(include=['object']).columns

# Paso 2: Limpiar las columnas de texto aplicando .str.strip(), .lower(), etc.
#for col in columnas_texto:
    # Reemplazar NaN por cadena vacía
 #   datos_pisosmadrid_detalles[col] = datos_pisosmadrid_detalles[col].fillna('')  # Reemplaza NaN por cadena vacía
    # Convertir a tipo str si es necesario
  #  datos_pisosmadrid_detalles[col] = datos_pisosmadrid_detalles[col].astype(str)
    # Limpiar espacios al principio y al final
   # datos_pisosmadrid_detalles[col] = datos_pisosmadrid_detalles[col].str.strip()

In [1275]:
datos_pisosmadrid_detalles["descripcion_larga"]=datos_pisosmadrid_detalles["descripcion_larga"].replace("Descripción:\n\t\t\t\t","",regex=True)
#datos_pisosmadrid_detalles["descripcion_larga"]=datos_pisosmadrid_detalles["descripcion_larga"].str.strip()
datos_pisosmadrid_detalles['descripcion_larga'] = (
    datos_pisosmadrid_detalles['descripcion_larga']
    .str.replace(r'\s+', ' ', regex=True)  # Reemplaza múltiples espacios por un solo espacio
    .str.strip()  # Elimina espacios al inicio y al final del texto
)

In [1276]:
pd.set_option('display.max_colwidth', None)

In [1277]:
datos_pisosmadrid_detalles["descripcion_larga"].head(10)

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [1278]:
datos_pisosmadrid_detalles["descripcion_larga"].isnull().sum()

0

In [1279]:
datos_pisosmadrid_normal["descripcion_larga"].isnull().sum()

0

In [1280]:
#elimino descripcion_larga de detalle

In [1281]:
#datos_pisosmadrid_detalles=datos_pisosmadrid_detalles.drop(columns="descripcion_larga")
#me quedo con el de detalle por que tiene todos los valores no se corta
datos_pisosmadrid_normal=datos_pisosmadrid_normal.drop(columns="descripcion_larga")

In [1282]:
#datos_pisosmadrid_normal["descripcion_larga"].head(10)

In [1283]:
# Encontrar columnas comunes
datos_pisosmadrid_detalles.columns.intersection(datos_pisosmadrid_normal.columns)


Index(['title', 'dormitorios', 'banos', 'm2_constr', 'tipologia', 'referencia',
       'precio'],
      dtype='object')

In [1284]:
datos_pisosmadrid_detalles["referencia"].value_counts(dropna=False)

referencia
4086665    1
3974110    1
3978041    1
3977655    1
3975253    1
          ..
4049210    1
4048885    1
4047995    1
4047935    1
3863841    1
Name: count, Length: 877, dtype: int64

In [1285]:
datos_pisosmadrid_normal["referencia"].value_counts(dropna=False)

referencia
4086665    1
3971814    1
3977655    1
3975253    1
3975227    1
          ..
4048885    1
4047995    1
4047935    1
4047505    1
3863841    1
Name: count, Length: 893, dtype: int64

columnas_comparar = ['title', 'precio',"dormitorios","banos","m2_constr","tipologia","referencia"                    ]

# Limitar ambos DataFrames a las primeras 1188 filas
datos_subset = datos_pisosmadrid_detalles[1:877]
datos2_subset = datos_pisosmadrid_normal[1:877]

# Comparar todas las columnas seleccionadas
coinciden = datos_subset[columnas_comparar] == (datos2_subset[columnas_comparar])
coinciden


In [1286]:
datos_pisosmadrid_detalles.columns

Index(['title', 'dormitorios', 'banos', 'm2_constr', 'descripcion_larga',
       'tipologia', 'ciudad', 'area', 'piscina', 'garaje', 'jardin',
       'referencia', 'fecha_publicacion', 'num_visitas', 'precio',
       'calefaccion', 'trastero', 'amueblado', 'chimenea', 'videoportero',
       'alarma', 'aire_acondicionado'],
      dtype='object')

In [1287]:
datos_pisosmadrid_detalles["area"].value_counts()

area
Centro                      58
Salamanca                   38
Malasaña                    37
Goya                        27
Arganzuela                  17
                            ..
Principe Pio                 1
Villaverde Alto              1
Acacias                      1
Cortes                       1
Montecarmelo - El Goloso     1
Name: count, Length: 147, dtype: int64

In [1288]:
datos_pisosmadrid_normal.isna().sum()

url              0
title            0
precio           0
dormitorios      0
banos            0
m2_constr        0
referencia       0
tipologia        0
planta         750
dtype: int64

In [1289]:
datos_pisosmadrid_detalles.isnull().sum()

title                   0
dormitorios             0
banos                   0
m2_constr               0
descripcion_larga       0
tipologia               0
ciudad                  0
area                  119
piscina                 0
garaje                  0
jardin                  0
referencia              0
fecha_publicacion       0
num_visitas             0
precio                  0
calefaccion           302
trastero              699
amueblado             718
chimenea              873
videoportero          595
alarma                831
aire_acondicionado    840
dtype: int64

In [1290]:
datos_pisosmadrid_detalles.head(2)

,title,dormitorios,banos,m2_constr,descripcion_larga,tipologia,ciudad,area,piscina,garaje,jardin,referencia,fecha_publicacion,num_visitas,precio,calefaccion,trastero,amueblado,chimenea,videoportero,alarma,aire_acondicionado
0,"Piso en venta en Ibiza, Madrid",3,1,70,"Descripción: Precioso piso reformado de 60m2 enfrente del Retiro, súper luminoso, todas las estancias con ventana exterior.~~Piso de 3 habitaciones, 1 baño con plato de ducha, salón, cocina totalmente equipada, interior con muchísima luz.~~Techos altos, suelos de tarima, ventanas blancas de climalit oscilobatientes, puertas lacadas en blanco, aire acondicionado en el salón.~~Ascensor, calefacción central, agua caliente por caldera de gas natural.~~Ubicación excelente en frente del Parque del Retiro.~~No dudes en venir a visitarlo!",Piso,Madrid,Ibiza,0,0,0,4086665,2024-10-24,4.0,520.000,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Piso en venta en Madrid,3,2,95,"~SERVICHECK INMOBILIARIA comercializa un precioso pisode 3 dormitorios, dos baños, con trastero, plaza de garaje y jardín en excelente ubicación.~¡Oportunidad única en una de las mejores zonas residenciales de Madrid!~~-Para una mejor experiencia visual entra a la visita en 3D"".~-(Para mejor comunicación enviar un WhatsApp) Para visitas presenciales, preguntar por Emiliano Correa al 602_250_283 (Para mejor comunicación enviar un WhatsApp)~~~El piso cuenta con una distribución cuidadosamente diseñada para brindarte la máxima comodidad y funcionalidad.Está en una tercera planta con ascensor, construido en 2007 y en muy buen estado de conservación.~~Dispone de 157 m² construidos en total: ~VIVIENDA 95 m2 ~APARCAMIENTO 10 m2 ~ALMACEN 5 m2 ~ELEMENTOS COMUNES 47 m2~~Distribuidos de manera funcional para aprovechar al máximo cada espacio.~~-Las características:~~-Hall de entrada.~-Salón comedor amplio y muy luminoso con vistas al interior de la finca, ideal para relajarse o disfrutar en familia.~-Cocina totalmente equipada con acceso a una terraza tendedero.~-Tres dormitorios con armarios empotrados, perfectos para mantener todo en orden.~-Dos cuartos de baño, uno con plato de ducha y otro en suite con bañera.~-Armario empotrado adicional en el pasillo, para mayor espacio de almacenamiento.~~La vivienda cuenta con calefacción individual por gas natural, garantizando confort en los meses de invierno y ""la preinstalación del aire acondicionado"". Además, se incluye una plaza de garaje y un trastero en el precio, brindando soluciones prácticas para almacenamiento adicional.~~El edificio cuenta con una amplia zona ajardinada, perfecta para pasear o disfrutar del aire libre sin salir de casa. ~Además, su ubicación es excelente, frente al Centro Deportivo Municipal Juan de Dios Román y a solo 7 minutos caminando del metro Las Suertes, con fácil acceso a la M45 y la A3. Cerca de todo tipo de servicios: centro de salud, colegios, guarderías, y el Centro Comercial La Gavia.~~-La ubicación: Calle de José Gutiérrez Maroto, Ensanche de VallecasBarrio Ensanche de Vallecas - La Gavia.Madrid.~~Una oportunidad ideal tanto para vivienda habitual como para inversión en una zona residencial tranquila y bien comunicada.",Piso,Madrid,NaN,0,1,1,4086544,2024-10-24,2.0,329.000,1.0,1.0,1.0,NaN,1.0,NaN,NaN


In [1291]:
datos_pisosmadrid_detalles.shape

(877, 22)

# Aqui guardamos el nombre de la columna objetivo en una variable
columna_a_clasificar= "barrio/localidad/distrito"

# Creamos la funcion que clasificara nombres en las columnas nuevas y tomara los valores de la columna target
def clasificar_barrio_distrito(datos_pisosmadrid_detalles, dbcp_ordenado,columna_a_clasificar):
    distritos= dbcp_ordenado["nombre_distrito"].tolist()
    barrios=   dbcp_ordenado["nombre_barrio"].tolist()

    # Creamos las nuevas columnas y las llenamos de nulos 
    datos_pisosmadrid_detalles["barrios"]   =np.nan 
    datos_pisosmadrid_detalles["distrito"]  =np.nan 
    datos_pisosmadrid_detalles["localidad"] =np.nan 

    for idx, valor in datos_pisosmadrid_detalles[columna_a_clasificar].items():
        if valor in distritos:
            datos_pisosmadrid_detalles.at[idx,"distrito" ]= valor
        elif valor in barrios:
            datos_pisosmadrid_detalles.at[idx,"barrios"   ]= valor 
        else:
            datos_pisosmadrid_detalles.at[idx,"localidad"]=valor
    datos_pisosmadrid_detalles[columna_a_clasificar]= np.nan

clasificar_barrio_distrito(datos_pisosmadrid_detalles = datos_pisosmadrid_detalles,dbcp_ordenado=dbcp_ordenado,columna_a_clasificar="barrio/localidad/distrito")


# silvia merge antes de continuar para tener en todas las columnas paa busccar barrio y distrito en el mismo dataframe

In [1292]:
datos_pisosmadrid_normal.shape, datos_pisosmadrid_detalles.shape

((893, 9), (877, 22))

In [1293]:
datos_pisosmadrid_detalles["title"]

0                          Piso en venta en Ibiza, Madrid
1                                 Piso en venta en Madrid
2                     Piso en venta en Arganzuela, Madrid
3                    Piso en venta en Los Angeles, Madrid
4                                 Piso en venta en Madrid
                              ...                        
1178                Piso en venta en Pueblo Nuevo, Madrid
1179                Apartamento en venta en Ibiza, Madrid
1180                    Piso en venta en Malasaña, Madrid
1182    Piso en venta en Montecarmelo - El Goloso, Madrid
1185                              Piso en venta en Madrid
Name: title, Length: 877, dtype: object

In [1294]:
columnas_eliminar=["title","precio","dormitorios","banos","m2_constr","tipologia","ciudad"]
datos_pisosmadrid_detalles=datos_pisosmadrid_detalles.drop(columns=columnas_eliminar)


In [1295]:
datos_pisosmadrid_detalles.reset_index(drop=True)
datos_pisosmadrid_normal.reset_index(drop=True)

,url,title,precio,dormitorios,banos,m2_constr,referencia,tipologia,planta
0,https://www.pisosmadrid.com.es/propiedad/4086665-piso-en-venta-en-ibiza-madrid,"Piso en venta en Ibiza, Madrid",520.000,3,1,70,4086665,Piso,NaN
1,https://www.pisosmadrid.com.es/propiedad/4086544-piso-en-venta-en-madrid,Piso en venta en Madrid,329.000,3,2,95,4086544,Piso,NaN
2,https://www.pisosmadrid.com.es/propiedad/4086503-piso-en-venta-en-arganzuela-madrid,"Piso en venta en Arganzuela, Madrid",740.000,4,3,158,4086503,Piso,NaN
3,https://www.pisosmadrid.com.es/propiedad/4086495-piso-en-venta-en-los-angeles-madrid,"Piso en venta en Los Angeles, Madrid",70.256,3,1,57,4086495,Piso,NaN
4,https://www.pisosmadrid.com.es/propiedad/4086493-piso-en-venta-en-madrid,Piso en venta en Madrid,97.920,2,1,50,4086493,Piso,NaN
...,...,...,...,...,...,...,...,...,...
888,https://www.pisosmadrid.com.es/propiedad/3865903-piso-en-venta-en-pueblo-nuevo-madrid,"Piso en venta en Pueblo Nuevo, Madrid",185.000,1 dormitorio,1,75,3865903,Piso,bajo
889,https://www.pisosmadrid.com.es/propiedad/3866605-apartamento-en-venta-en-ibiza-madrid,"Apartamento en venta en Ibiza, Madrid",1.329.000,3,2,173,3866605,Apartamento,segunda planta
890,https://www.pisosmadrid.com.es/propiedad/3865902-piso-en-venta-en-malasana-madrid,"Piso en venta en Malasaña, Madrid",760.000,5,3,170,3865902,Piso,NaN
891,https://www.pisosmadrid.com.es/propiedad/3865872-piso-en-venta-en-montecarmelo---el-goloso-madrid,"Piso en venta en Montecarmelo - El Goloso, Madrid",825.000,4,2,142,3865872,Piso,NaN


In [1296]:
datos_pisosmadrid_normal.shape,datos_pisosmadrid_detalles.shape

((893, 9), (877, 15))

In [1297]:
# merge de pisosmadrid y redpiso 
dfpisosmadrid= pd.merge(datos_pisosmadrid_normal,datos_pisosmadrid_detalles,on=["referencia"],how="outer")

In [1298]:
dfpisosmadrid.shape

(893, 23)

In [1299]:
dfpisosmadrid.columns

Index(['url', 'title', 'precio', 'dormitorios', 'banos', 'm2_constr',
       'referencia', 'tipologia', 'planta', 'descripcion_larga', 'area',
       'piscina', 'garaje', 'jardin', 'fecha_publicacion', 'num_visitas',
       'calefaccion', 'trastero', 'amueblado', 'chimenea', 'videoportero',
       'alarma', 'aire_acondicionado'],
      dtype='object')

In [1300]:
dfpisosmadrid.reset_index(drop=True)

,url,title,precio,dormitorios,banos,m2_constr,referencia,tipologia,planta,descripcion_larga,area,piscina,garaje,jardin,fecha_publicacion,num_visitas,calefaccion,trastero,amueblado,chimenea,videoportero,alarma,aire_acondicionado
0,https://www.pisosmadrid.com.es/propiedad/3863841-piso-en-venta-en-madrid,Piso en venta en Madrid,60.000,3,1,54,3863841,Piso,NaN,"PISO EN MADRID. ~Vivienda situada en Avenida Palomeras, en Madrid. VIVIENDA CON INCIDENCIAS: INMUEBLE SIN POSESIÓN. ~~El estado de ocupado y/o los impedimentos para que el inmueble pueda ser visitado libremente por los técnicos impiden su tasación oficial conforme exige la Orden ECO/805/2003, de 27 de marzo, por lo que el inmueble, mientras perdure la situación, no podrá ser hipotecado, con lo cual la compra sería al CONTADO. ~~Este inmueble ha sido adquirido por el Vendedor, pero aún no se ha tomado la posesión, al estar OCUPADO. El Adquirente asumirá; la situación física y estado de conservación; la situación urbanística; las discrepancias entre la realidad física y la realidad registral y catastral o en la cabida del Inmueble; la adecuación registral y catastral del Inmueble; y los tramites que conlleven la obtención de la Posesión Judicial del Inmueble y/o posibles acuerdos extrajudiciales con el ocupante en el caso de haberlo.~~Honorarios de gestión no incluidos en el precio de compra.~~VENDE SINERGIA INVESTMENT – 613 508 666 – 854 53 53 59~~www.inmobiliariasinergia.es",NaN,0.0,0.0,0.0,2024-05-17,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.pisosmadrid.com.es/propiedad/3865872-piso-en-venta-en-montecarmelo---el-goloso-madrid,"Piso en venta en Montecarmelo - El Goloso, Madrid",825.000,4,2,142,3865872,Piso,NaN,"600 Aprovecha esta oportunidad de compra de esta exclusiva vivienda de 4 espaciosos dormitorios en la mejor zona de Madrid y en comunidad privada con dos piscinas, una de ellas exclusiva para que disfruten los más pequeños. Con toda la carpintería renovada, la vivienda cambia el color marrón original en puertas y rodapiés por el blanco, ganando una luminosidad que se expande por toda la vivienda. Recién pintada cuenta también con ventanas oscilobatientes de máxima calidad recientemente instaladas, con rotura de puente térmico para aislar mejor la vivienda y ganar eficiencia energética, consiguiendo un ahorro en costes y mejora del medio ambiente. Las persianas eléctricas se acaban de incorporar a la vivienda, cada una con su propio mando, y además cuenta con uno universal para todas ellas (a excepción del dormitorio principal). El suelo de madera brilla recién barnizado. Qué maravilla poder disfrutar de esta vivienda de 4 dormitorios, probablemente de los más grandes y espaciosos de todo Montecarmelo. La cocina, completamente equipada, cuenta además con un amplio office para ubicar lavadora y secadora. Cuenta con aire acondicionado y bomba de calor, además de gas natural. Y por supuesto con una plaza de garaje y trastero para que puedas guardar las bicicletas que usarás por todos los km de carril bici que conecta Montecarmelo con el resto de Madrid. En la mejor zona de Montecarmelo, muy cerquita del centro comercial, con salida a una tranquila calle peatonal, ideal para niños, y a un paso del acceso a M30 y M40, cuenta además con transporte público y otros servicios como colegios públicos, concertados y privados, entre ellos el Alemán; varios supermercados, gimnasios, la magnífica Ciudad de la Raqueta y el Centro Nacional de Golf. Además en breve entre estos servicios se incluirán un nuevo centro de salud y una biblioteca que ya están en construcción. ¿Se puede pedir más?",Montecarmelo - El Goloso,1.0,1.0,1.0,2024-05-18,15.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
2,https://www.pisosmadrid.com.es/propiedad/3865902-piso-en-venta-en-malasana-madrid,"Piso en venta en Malasaña, Madrid",760.000,5,3,170,3865902,Piso,NaN,"104 ¡Increíble oportunidad de inversión en el corazón de Madrid! Este espectacular inmueble de 170 metros cuadrados se encuentra ubicado en la prestigiosa Calle B

In [1301]:
dfpisosmadrid.shape

(893, 23)

In [1302]:
dfpisosmadrid.columns

Index(['url', 'title', 'precio', 'dormitorios', 'banos', 'm2_constr',
       'referencia', 'tipologia', 'planta', 'descripcion_larga', 'area',
       'piscina', 'garaje', 'jardin', 'fecha_publicacion', 'num_visitas',
       'calefaccion', 'trastero', 'amueblado', 'chimenea', 'videoportero',
       'alarma', 'aire_acondicionado'],
      dtype='object')

# silvia, empiezo a modificar distrito sobre el dataframe unido

In [1303]:
nuevas_columnas= ["barrio","cod_barrio","distrito","cod_distrito","codigo_Postal"]
dfpisosmadrid[nuevas_columnas]=np.nan

# Cambiar el tipo de dato de las columnas "barrio" y "distrito" a object
dfpisosmadrid["barrio"] = dfpisosmadrid["barrio"].astype(object)
dfpisosmadrid["distrito"] = dfpisosmadrid["distrito"].astype(object)
dfpisosmadrid["cod_barrio"] = dfpisosmadrid["cod_barrio"].astype(object)
dfpisosmadrid["cod_distrito"] = dfpisosmadrid["cod_distrito"].astype(object)
dfpisosmadrid["codigo_Postal"] = dfpisosmadrid["codigo_Postal"].astype(object)
dfbarrios.loc[:, 'distrito_n'] = [normalize_string(valor.strip().replace(" ", "")) for valor in dfbarrios.loc[:, 'distrito']]
dfbarrios.loc[:, 'barrio_n'] = [normalize_string(valor.strip().replace(" ", "")) for valor in dfbarrios.loc[:, 'barrio']]

In [1304]:
# Asignar valor específico si la URL coincide
dfpisosmadrid.loc[dfpisosmadrid["url"] == "https://www.pisosmadrid.com.es/propiedad/3912876-piso-en-venta-en-malasana-madrid", "m2_constr"] = 131

dfpisosmadrid.loc[dfpisosmadrid["url"] == "https://www.pisosmadrid.com.es/propiedad/4056174-apartamento-en-venta-en-centro-madrid", "m2_constr"] = 100

In [1305]:
url_a_eliminar = ['https://www.pisosmadrid.com.es/propiedad/4084626-negocio-en-venta-en-carabanchel-madrid','https://www.pisosmadrid.com.es/propiedad/4002776-piso-en-venta-en-madrid',
'https://www.pisosmadrid.com.es/propiedad/3867155-piso-en-venta-en-carabanchel-madrid',
'https://www.pisosmadrid.com.es/propiedad/3964989-piso-en-venta-en-vicalvaro-madrid',
'https://www.pisosmadrid.com.es/propiedad/3937024-duplex-en-venta-en-usera-madrid',
'https://www.pisosmadrid.com.es/propiedad/3867144-piso-en-venta-en-vicalvaro-madrid',
'https://www.pisosmadrid.com.es/propiedad/4045767-piso-en-venta-en-puente-de-vallecas-madrid',
'https://www.pisosmadrid.com.es/propiedad/3932188-piso-en-venta-en-puente-de-vallecas-madrid',
'https://www.pisosmadrid.com.es/propiedad/3937690-duplex-en-venta-en-usera-madrid',
'https://www.pisosmadrid.com.es/propiedad/3867132-piso-en-venta-en-puente-de-vallecas-madrid',
'https://www.pisosmadrid.com.es/propiedad/4045772-piso-en-venta-en-usera-madrid','https://www.pisosmadrid.com.es/propiedad/4066464-piso-en-venta-en-madrid',
'https://www.pisosmadrid.com.es/propiedad/4033699-piso-en-venta-en-san-blas-madrid','https://www.pisosmadrid.com.es/propiedad/4024490-piso-en-venta-en-madrid',
'https://www.pisos.com/comprar/piso-fontarron-47524209698_100600/','https://www.pisosmadrid.com.es/propiedad/3907461-piso-en-venta-en-madrid',
"https://www.pisosmadrid.com.es/propiedad/3930386-piso-en-venta-en-madrid","https://www.pisosmadrid.com.es/propiedad/3967140-piso-en-venta-en-madrid"
,"https://www.pisosmadrid.com.es/propiedad/3986133-piso-en-venta-en-madrid","https://www.pisosmadrid.com.es/propiedad/4010211-piso-en-venta-en-madrid","https://www.pisosmadrid.com.es/propiedad/4030101-piso-en-venta-en-centro-madrid"]



# Eliminar filas donde 'url' coincide con alguna en la lista 'url_a_eliminar'
dfpisosmadrid = dfpisosmadrid[~dfpisosmadrid['url'].isin(url_a_eliminar)]

In [1306]:
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3895829-piso-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','nuda_propiedad']] = ['Puente de Vallecas',13,'Palomeras Bajas',133,1]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3890362-piso-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','nuda_propiedad']] = ['Chamberí',7,'Almagro',74,1]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3895577-piso-en-venta-en-campo-de-las-naciones-madrid'), ['distrito','cod_distrito','barrio','cod_barrio']] = ['Barajas',21,'Corralejo',215]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3895829-piso-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','nuda_propiedad']] = ['Puente de Vallecas',13,'San Diego',132,1]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3895852-piso-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','nuda_propiedad','tiene_piscina']] = ['Fuencarral-El Pardo',8,'El Pardo',81,1,0]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3906133-piso-en-venta-en-madrid'), ['letrace','distrito','cod_distrito','barrio','cod_barrio','nuda_propiedad','tiene_piscina','garaje','trastero','estado']] = ['B','Tetuán',6,'Valdeacederas',65,0,1,1,1,'obra nueva']
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3906149-piso-en-venta-en-madrid'), ['letrace','distrito','cod_distrito','barrio','cod_barrio','nuda_propiedad','tiene_piscina','garaje','trastero','estado','planta']] = ['B','Tetuán',6,'Valdeacederas',65,0,1,1,1,'obra nueva',1]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3909307-piso-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','gastos_comunidad','calefaccion']] = ['Puente de Vallecas',13,'Portazgo',135,50,'radiadores gas natural']
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4086544-piso-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','planta','ascensor','calefaccion','ano_construccion','garaje','trastero']] = ['Villa de Vallecas',18,'Ensanche de Vallecas',183, 3, 1,'radiadores gas natural',2007, 1, 1]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3912746-adosado-en-venta-en-valdebebas-madrid'), ['distrito','cod_distrito','barrio','cod_barrio']] = ['Hortaleza',16,'Valdefuentes',167]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3913534-atico-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','terraza','porteria','tiene_piscina','garaje','trastero']] = ['San Blas-Canillejas',20,'Salvador',208,1,'todo el dia',1,1,1]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3914118-adosado-en-venta-en-puerta-de-hierro-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','terraza','porteria','tiene_piscina','garaje']] = ['Moncloa-Aravaca',9,'Ciudad Universitaria',93,1,'todo el dia',1,1]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3926465-piso-en-venta-en-malasana-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','balcon','cocina']] = ['Centro',1,'Palacio',11,1,'amueblada y equipada']
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3941614-piso-en-venta-en-malasana-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','balcon','estado','garaje','precio','terraza']] = ['Centro',1,'Universidad',15,0,'reformado',0,295000,0]
#dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3930386-piso-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','tiene_piscina','garaje','trastero','inmueble_ingresos','estado','ascensor']] = ['San Blas-Canillejas',20,'Simancas',201,0,0,0,1,'buen estado',1]

dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('3930386-piso-en-venta-en-madrid'), 
         ['distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'tiene_piscina', 'garaje', 'trastero', 'inmueble_ingresos', 'estado', 'ascensor']] = ['San Blas-Canillejas', 20, 'Simancas', 201, 0, 0, 0, 1, 'buen estado', 1]

dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3907116-piso-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','ano_construccion','ascensor','calefaccion','gastos_comunidad']] = ['Puente de Vallecas',13,'San Diego',132,1997,1,'gas natural',55]


dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3952091-piso-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','ano_construccion','estado','terraza']] = ['Villa de Vallecas',18,'Casco Histórico de Vallecas',181,1965,'para reformar',0]

dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3955423-piso-en-venta-en-gran-via-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','estado','terraza','portero','aire_acondicionado','planta','balcon','exterior','calefaccion','cocina']] = ['Centro',1,'Cortes',13,'reformado',0,1,'por splits',7,1,1,'central por gas','equipada e integrada']

dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3980091-piso-en-venta-en-arturo-soria-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','estado','terraza','portero','garaje','trastero','aire_acondicionado','exterior','ascensor']] = ['Ciudad Lineal',13,'Quintana',153,'reformado',1,1,1,1,1,1,1]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3984516-piso-en-venta-en-las-mercedes-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','estado']] = ['San Blas-Canillejas',20,'Rejas',206,'obra nueva']

dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3984517-piso-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','planta','ascensor','terraza']] = ['Villa de Vallecas',18,'Casco Histórico de Vallecas',181,7,1,1]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3993106-piso-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','piscina','porteria','armarios','ascensor','terraza','calefaccion','estado']] = ['Ciudad Lineal',15,'San Juan Bautista',156,1,1,1,1,1,'central','reformado']

dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3999010-piso-en-venta-en-malasana-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','piscina','estado']] = ['Centro',1,'Universidad',15,0,'reformado']
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4002199-piso-en-venta-en-puerta-de-hierro-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','terraza','planta','gastos_comunidad']] = ['Moncloa-Aravaca',9,'Ciudad Universitaria',93,1,1,211]
#pongo nuda_propiedad aunque es subasta
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4009926-piso-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','nuda_propiedad']] = ['Puente de Vallecas',13,'Portazgo',135,1]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4061729-piso-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','trastero','garaje','nuda_propiedad']] = ['Villa de Vallecas',18,'Ensanche de Vallecas',183,1,1,1]

dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4018244-atico-en-venta-en-principe-pio-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','terraza','cocina']] = ['Moncloa-Aravaca',9,'Ciudad Universitaria',91,1,'independiente con zona office']
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4019953-piso-en-venta-en-gran-via-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','terraza']] = ['Centro',1,'Cortes',13,1]

dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4019754-piso-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','estado','planta','plantas_edificio','trastero','videoportero','armarios','exterior']] = ['Chamberí',7,'Arapiles',72,'reformado',5,6,1,1,1,1]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4021421-piso-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','piscina','estado','lujo','balcon','exterior','trastero','cocina','orientacion_este']] = ['Centro',1,'Universidad',15,0,'reformado',1,1,1,1,'abierta',1]

dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4022055-piso-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','estado','terraza']] = ['Salamanca',4,'Goya',42,'obra nueva',1]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4027823-piso-en-venta-en-malasana-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','piscina','estado','armarios','ascensor','interior','cocina']] = ['Centro',1,'Universidad',15,0,'reformado',1,1,1,'equipada']
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4038755-apartamento-en-venta-en-valdebebas-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','inmueble_ingresos','amueblado']] = ['Hortaleza',16,'Valdefuentes',166,1,1]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4045589-piso-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','planta','exterior','orietacion_oeste','cocina','estado']] = ['Moncloa-Aravaca',9,'Arguelles',92,3,1,1,'independiente equipada','reformado']

dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4049261-piso-en-venta-en-malasana-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','planta','estado','ano_construccion','orientacion_sur']] = ['Centro',1,'Universidad',15,4,'para reformar',1951,1]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4049262-piso-en-venta-en-malasana-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','planta','estado','ano_construccion','orientacion_sur','interior']] = ['Centro',1,'Universidad',15,3,'para reformar',1951,1,1]

dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4049998-estudio-en-venta-en-valdebebas-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','piscina','amueblado','estado']] = ['Hortaleza',16,'Valdefuentes',166,1,1,'buen estado']
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4055061-estudio-en-venta-en-lavapies-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','planta','cocina']] = ['Centro',1,'Embajadores',12,0,'americana']
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4058252-piso-en-venta-en-ensanche-vallecas-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','piscina','trastero','garaje','cocina','calefaccion','aire_acondicionado','porteria']] = ['Villa de Vallecas',18,'Ensanche de Vallecas',183,1,1,1,'independiente','gas natural',1,1]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4060837-piso-en-venta-en-campo-de-las-naciones-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','piscina','trastero','garaje','cocina','calefaccion','aire_acondicionado','estado']] = ['Hortaleza',16,'Palomas',161,0,1,1,'independiente','gas natural',1,'buen estado']

dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4061729-piso-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','trastero','garaje']] = ['Villa de Vallecas',18,'Ensanche de Vallecas',183,1,1]

dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4063624-piso-en-venta-en-malasana-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','inmueble_ingresos']] = ['Centro',1,'Universidad',15,1]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4065057-piso-en-venta-en-malasana-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','amueblado','balcon','videoportero','calefaccion','aire_acondicionado']] = ['Centro',1,'Universidad',15,1,1,1,'aire',1]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4070178-piso-en-venta-en-malasana-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','balcon','exterior','planta','gastos_comunidad','aire_acondicionado','porteria']] = ['Centro',1,'Universidad',15,1,1,1,85,'bomba frio calor','media jornada']
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4086303-piso-en-venta-en-malasana-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','estado','precio','calefaccion']] = ['Centro',1,'Universidad',15,'para reformar',1250000,'central']


dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4063625-piso-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','inmueble_ingresos']] = ['Latina',10,'Lucero',103,1]

dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4068880-piso-en-venta-en-las-tablas-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','trastero','garaje','piscina','cocina']] = ['Fuencarral el pardo ',8,'Valverde',86,1,1,1,'independiente']
#https://www.pisosmadrid.com.es/propiedad/3890778-piso-en-venta-en-madrid

dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4074173-piso-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','planta']] = ['Salamanca',4,'Guindalera',44,0]

dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4070179-atico-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','terraza','orientacion_sur','porteria','calefaccion']] = ['Chamberí',7,'Vallehermoso',76,1,1,1,'individual gas natural']
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4077780-piso-en-venta-en-costillares-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','planta','exterior','armarios','cocina','garaje','trastero','piscina','porteria']] = ['Ciudad Lineal',15,'Costillares',159,8,1,1,'independiente equipada',2,1,1,1]

dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4077645-apartamento-en-venta-en-san-blas-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','piscina','exterior','armarios','cocina','porteria']] = ['San Blas-Canillejas',20,'Rosas',205,1,1,1,'independiente equipada',1]

dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4079312-piso-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','estado','ano_construccion','amueblado','cocina']] = ['Centro',1,'Cortes',13,'reformado',1890,1,'amueblada equipada']
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4079795-adosado-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','estado','garaje','trastero','aire_ acondicionado']] = ['Villaverde',17,'Villaverde Alto',171,'reformado',1,1,1]


dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4085127-apartamento-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','tipologia','garaje','trastero','porteria','piscina','cocina']] = ['Arganzuela',2,'Acacias',22,'Estudio',1,1,1,1,'independiente']


dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4086493-piso-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','nuda_propiedad']] = ['San Blas-Canillejas',20,'Arcos',204,1]

C:\Users\extas\AppData\Local\Temp\ipykernel_12100\478194640.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'radiadores gas natural' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3909307-piso-en-venta-en-madrid'), ['distrito','cod_distrito','barrio','cod_barrio','gastos_comunidad','calefaccion']] = ['Puente de Vallecas',13,'Portazgo',135,50,'radiadores gas natural']
C:\Users\extas\AppData\Local\Temp\ipykernel_12100\478194640.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'por splits' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3955423-piso-

In [1307]:

dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4010194-piso-en-venta-en-madrid'), ['m2_constr','distrito','cod_distrito','barrio','cod_barrio','inmueble_ingresps']] = [47,'Puente de Vallecas',13,'San Diego',132,1]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4010211-piso-en-venta-en-madrid'), ['m2_constr']] = 47
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/4056174-apartamento-en-venta-en-centro-madrid'), ['m2_constr']] = 100
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3908548-piso-en-venta-en-madrid'), ['m2_constr']] = 302
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3908547-piso-en-venta-en-madrid'), ['m2_constr']] = 316
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3908546-piso-en-venta-en-madrid'), ['m2_constr','planta']] = [185,1]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3908545-piso-en-venta-en-madrid'), ['m2_constr','planta']] = [311,3]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3908523-piso-en-venta-en-madrid'), ['m2_constr','ascensor']] = [348,1]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3908520-piso-en-venta-en-madrid'), ['m2_constr','terraza','trastero']] = [263,1,1]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3908448-chalet-en-venta-en-madrid'), ['m2_constr']] = 1800
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3907116-piso-en-venta-en-madrid'), ['m2_constr','planta','ascensor']] = [60,2,1]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.indomio.es/anuncios/96517417/'), ['m2_constr']] = 397
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisos.com/comprar/piso-villaverde_san_andres28021-4918181785_109700/'), ['m2_constr','planta','estado']] = [88,1,'obra nueva']
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisos.com/comprar/piso-villaverde_san_andres28021-4915060649_109700/'), ['m2_constr','planta','estado']] = [109,0,'obra nueva']
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.pisosmadrid.com.es/propiedad/3907141-piso-en-venta-en-madrid'), ['cod_barrio','barrio','cod_distrito','distrito','m2_constr','planta','estado','num_plantas','tiene_piscina','trastero','calefaccion']] = [74,'Almagro',7,'Chamberí',184,1,'reformado',7,1,1,'aerotermia suelo radiante']
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.indomio.es/anuncios/94926741/'), ['cod_barrio','barrio','cod_distrito','distrito','m2_constr','planta','tiene_piscina','trastero','inmueble_ingresos']] = [101,'Los Cármenes',10,'Latina',80,1,0,0,1]
dfpisosmadrid.loc[dfpisosmadrid['url'].str.contains('https://www.indomio.es/anuncios/95261315/'), ['m2_constr','tiene_piscina','Lujo']] = [271,1,1]


urlsnuevo=['https://www.pisos.com/comprar/apartamento-adelfas28007-45835099732_101800/','https://www.pisos.com/comprar/apartamento-adelfas28007-45869202772_101800/',
'https://www.pisos.com/comprar/apartamento-adelfas28007-45912206979_101800/',
'https://www.pisos.com/comprar/apartamento-adelfas28007-45919489438_101800/',
'https://www.pisos.com/comprar/apartamento-adelfas28007-46667942689_101800/',
'https://www.pisos.com/comprar/apartamento-adelfas28007-46731956156_101800/']
#les asignamos una planta 5, los aticos son mas caros 
# Actualizar las filas correspondientes usando isin
dfpisosmadrid.loc[dfpisosmadrid['url'].isin(urlsnuevo), ['estado', 'trastero', 'tiene_piscina', 'garaje', 'planta']] = ['Obra nueva', 1, 1, 1, 5]






In [1308]:


dfcalles.loc[:, 'distrito_n'] = [normalize_string(valor.strip().replace(" ", "")) for valor in dfcalles.loc[:, 'Nombre del distrito']]
dfcalles.loc[:, 'barrio_n'] = [normalize_string(valor.strip().replace(" ", "")) for valor in dfcalles.loc[:, 'Nombre del barrio']]

# Normalizar y limpiar la columna 'Nombre de la vía'
dfcalles['calle_n'] = dfcalles['Nombre de la vía'].astype(str).str.strip().str.replace(",", "").str.replace("Calle", "").str.replace("Avenida", "").str.replace("Plaza", "").str.strip().apply(normalize_string).str.replace(" ", "")

def clasificar_distrito_y_barrio_calle(df, dfcalles):
    # Recorrer las filas del dataframe
    for idx, fila in df.iterrows():
        # Verificar si la columna "barrio" es nula
        if pd.isna(fila["barrio"]):
            # Recorrer las columnas en orden de prioridad: calle
            for columna in ['calle']:
                valor = fila[columna]                
                # Verificamos que el valor no sea nulo
                if pd.notna(valor):
                    valor_normalizado = normalize_string(valor.strip().replace("Calle", "").replace("Avenida", "").replace("Plaza", "").replace(" ", "").replace(",", ""))
                    print(valor_normalizado)
                    # Verificamos si el valor normalizado contiene el nombre de una calle
                    #calle_encontrada = next((calle for calle in dfcalles["calle_n"] if calle in valor_normalizado), None)
                    # Verificamos si el valor normalizado es exactamente igual al nombre de una calle
                    calle_encontrada = next((calle for calle in dfcalles["calle_n"] if calle == valor_normalizado), None)

                    if calle_encontrada:
                        print(calle_encontrada)
                        # Buscar los valores cod_barrio, distrito y cod_distrito en dfcalles
                        result = dfcalles.query(f"calle_n == '{calle_encontrada}'")
                        if not result.empty:
                            
                            barrio = result["Nombre del barrio"].values[0]
                            cod_distrito = result["Codigo de distrito"].values[0]
                            cod_barrio = result["Codigo de barrio"].values[0]
                            cod_barrio1=cod_distrito & cod_barrio
                            codigo_Postal = result["Codigo postal"].values[0]
                            distrito_result = result["Nombre del distrito"].values[0]
                            print(cod_barrio1)
                            # Asignar los valores encontrados a las columnas correspondientes
                            df.at[idx, "barrio"] = barrio
                            df.at[idx, "cod_barrio"] = cod_barrio1
                            df.at[idx, "cod_distrito"] = cod_distrito
                            df.at[idx, "distrito"] = distrito_result
                            df.at[idx, "codigo_Postal"] = codigo_Postal
                    

                        
    return df


In [1309]:
dfpisosmadrid.query("cod_barrio.isnull()")[["url","barrio","distrito","cod_distrito","tipologia"]]

,url,barrio,distrito,cod_distrito,tipologia
0,https://www.pisosmadrid.com.es/propiedad/3863841-piso-en-venta-en-madrid,NaN,NaN,NaN,Piso
1,https://www.pisosmadrid.com.es/propiedad/3865872-piso-en-venta-en-montecarmelo---el-goloso-madrid,NaN,NaN,NaN,Piso
2,https://www.pisosmadrid.com.es/propiedad/3865902-piso-en-venta-en-malasana-madrid,NaN,NaN,NaN,Piso
3,https://www.pisosmadrid.com.es/propiedad/3865903-piso-en-venta-en-pueblo-nuevo-madrid,NaN,NaN,NaN,Piso
4,https://www.pisosmadrid.com.es/propiedad/3866311-piso-en-venta-en-villaverde-madrid,NaN,NaN,NaN,Piso
...,...,...,...,...,...
885,https://www.pisosmadrid.com.es/propiedad/4086130-piso-en-venta-en-madrid-madrid,NaN,NaN,NaN,Piso
887,https://www.pisosmadrid.com.es/propiedad/4086488-piso-en-venta-en-san-diego-madrid,NaN,NaN,NaN,Piso
889,https://www.pisosmadrid.com.es/propiedad/4086495-piso-en-venta-en-los-angeles-madrid,NaN,NaN,NaN,Piso
890,https://www.pisosmadrid.com.es/propiedad/4086503-piso-en-venta-en-arganzuela-madrid,NaN,NaN,NaN,Piso


In [1310]:
clasificar_distrito_y_barrio_por_columnas(dfpisosmadrid, dfbarrios,'title')

In [1311]:
#clasificar_distrito_y_barrio_por_columnas(dfpisosmadrid, dfbarrios,'title')

#clasificar_distrito_y_barrio_por_columnas(dfpisosmadrid, dfbarrios,'url')

In [1312]:
#clasificar_distrito_y_barrio_por_columnas(dfpisosmadrid, dfbarrios,'area')

In [1313]:
#clasificar_distrito_y_barrio_por_columnas(dfpisosmadrid, dfbarrios,'area')
clasificar_distrito_y_barrio_por_columnas(dfpisosmadrid, dfbarrios,'descripcion_larga')

In [1314]:
#dfpisosmadrid["calle"] = dfpisosmadrid["descripcion_larga"].apply(extraer_calle_1palabra)

In [1315]:
dfpisosmadrid["calle"]=np.nan
# Aplicar extraer_calle solo si "calle" es nulo
dfpisosmadrid.loc[dfpisosmadrid["calle"].isnull(), "calle"] = dfpisosmadrid["descripcion_larga"].apply(extraer_calle_1palabra)

#dfpisosmadrid.loc[dfpisosmadrid["calle"].isnull(), "calle"] = dfpisosmadrid["descripcion_larga"].apply(extraer_calle)
dfpisosmadrid["calle"]=dfpisosmadrid["calle"].str.strip()

C:\Users\extas\AppData\Local\Temp\ipykernel_12100\2975251919.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['Palomeras,' 'de' 'Barco,' 'Alcalá' None 'San' None 'Ortega' 'Velázquez'
 None 'Ronda' 'Lenceros,' None None 'Embajadores,' 'Santa' None
 'Elíptica,' None None 'Calamina,' 'de' 'Cardenal' None None None
 'Antonio' 'Cardenal' None 'de' None None None None None 'Manuel' 'DE'
 None None 'Sierra' None 'La' 'Encomienda' None None None None 'del' None
 'América);' None None 'en' 'Reina' 'Diario' 'Villajimena,' None None None
 None None None None None 'Campiña,' None None None None None 'de'
 'Cartagena,' 'Pinos' 'Pinos' 'Blasco' 'Valdesaz,' 'María' 'Pinazo,' 'Sor'
 'Godella,' 'Mariano' 'Alhambra,' 'Gallur,' 'Encajeras,' 'Puerto' None
 'Alzola,' 'Abardero,' None None None 'Mesoneros' None 'con' None None
 'mayor,' None None None 'Gaztambide.~Muy' 'Moquetas' None 'Moquetas' None
 'Génova.' 'cuenta' N

In [1316]:
# lamar a la función pasándole el dataframe y el dataframe de referencia

clasificar_distrito_y_barrio_calle(dfpisosmadrid, dfcalles)

palomeras
palomeras
1
calamina
calamina
1
antonio
antonio
4
cardenal
villajimena
villajimena
1
cartagena
cartagena
4
pinos
pinos
valdesaz
valdesaz
0
gallur
gallur
0
encajeras
encajeras
0
puerto
ortega
de
de
de
nuestra
juan
menorca.
barcelo
barcelo
0
de
arandano
arandano
4
de
de
claudio
garaje
castello
castello
0
cascorro.~~esta
castello
castello
0
de
bravo
bravo
0
alcala
alcala
0
de
jose
jose
4
y


,url,title,precio,dormitorios,banos,m2_constr,referencia,tipologia,planta,descripcion_larga,area,piscina,garaje,jardin,fecha_publicacion,num_visitas,calefaccion,trastero,amueblado,chimenea,videoportero,alarma,aire_acondicionado,barrio,cod_barrio,distrito,cod_distrito,codigo_Postal,nuda_propiedad,tiene_piscina,letrace,estado,gastos_comunidad,ascensor,ano_construccion,terraza,porteria,balcon,cocina,inmueble_ingresos,portero,exterior,armarios,plantas_edificio,lujo,orientacion_este,interior,orietacion_oeste,orientacion_sur,aire_ acondicionado,inmueble_ingresps,num_plantas,Lujo,calle
0,https://www.pisosmadrid.com.es/propiedad/3863841-piso-en-venta-en-madrid,Piso en venta en Madrid,60.000,3,1,54,3863841,Piso,NaN,"PISO EN MADRID. ~Vivienda situada en Avenida Palomeras, en Madrid. VIVIENDA CON INCIDENCIAS: INMUEBLE SIN POSESIÓN. ~~El estado de ocupado y/o los impedimentos para que el inmueble pueda ser visitado libremente por los técnicos impiden su tasación oficial conforme exige la Orden ECO/805/2003, de 27 de marzo, por lo que el inmueble, mientras perdure la situación, no podrá ser hipotecado, con lo cual la compra sería al CONTADO. ~~Este inmueble ha sido adquirido por el Vendedor, pero aún no se ha tomado la posesión, al estar OCUPADO. El Adquirente asumirá; la situación física y estado de conservación; la situación urbanística; las discrepancias entre la realidad física y la realidad registral y catastral o en la cabida del Inmueble; la adecuación registral y catastral del Inmueble; y los tramites que conlleven la obtención de la Posesión Judicial del Inmueble y/o posibles acuerdos extrajudiciales con el ocupante en el caso de haberlo.~~Honorarios de gestión no incluidos en el precio de compra.~~VENDE SINERGIA INVESTMENT – 613 508 666 – 854 53 53 59~~www.inmobiliariasinergia.es",NaN,0.0,0.0,0.0,2024-05-17,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PALOMERAS BAJAS,1,PUENTE DE VALLECAS,13,28018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Palomeras,"
1,https://www.pisosmadrid.com.es/propiedad/3865872-piso-en-venta-en-montecarmelo---el-goloso-madrid,"Piso en venta en Montecarmelo - El Goloso, Madrid",825.000,4,2,142,3865872,Piso,NaN,"600 Aprovecha esta oportunidad de compra de esta exclusiva vivienda de 4 espaciosos dormitorios en la mejor zona de Madrid y en comunidad privada con dos piscinas, una de ellas exclusiva para que disfruten los más pequeños. Con toda la carpintería renovada, la vivienda cambia el color marrón original en puertas y rodapiés por el blanco, ganando una luminosidad que se expande por toda la vivienda. Recién pintada cuenta también con ventanas oscilobatientes de máxima calidad recientemente instaladas, con rotura de puente térmico para aislar mejor la vivienda y ganar eficiencia energética, consiguiendo un ahorro en costes y mejora del medio ambiente. Las persianas eléctricas se acaban de incorporar a la vivienda, cada una con su propio mando, y además cuenta con uno universal para todas ellas (a excepción del dormitorio principal). El suelo de madera brilla recién barnizado. Qué maravilla poder disfrutar de esta vivienda de 4 dormitorios, probablemente de los más grandes y espaciosos de todo Montecarmelo. La cocina, completamente equipada, cuenta además con un amplio office para ubicar lavadora y secadora. Cuenta con aire acondicionado y bomba de calor, además de gas natural. Y por supuesto con una plaza de garaje y trastero para que puedas guardar las bicicletas que usarás por todos los km de carril bici que conecta Montecarmelo con el resto de Madrid. En la mejor zona de Montecarmelo, muy cerquita del centro comercial, con salida a una tranquila calle peatonal, ideal para niños, y a un paso del acceso a M30 y M40, cuenta además con transporte público y otros servicios como colegios públicos, concertados y privados, entre ellos el Alemán; varios supermercados, gimnasios, la magnífica Ciudad de la Raqueta y el Centro Nacional de Golf. Además en breve entre est

In [1317]:
dfpisosmadrid.query("barrio.isnull()")[["title","url","area","descripcion_larga","barrio","calle"]]

,title,url,area,descripcion_larga,barrio,calle
27,Piso en venta en Madrid,https://www.pisosmadrid.com.es/propiedad/3872239-piso-en-venta-en-madrid,NaN,"DP2020 vende piso en edificio residencial con ascensor, ubicado en la localidad de Madrid~Consta de una superficie de 102m² bien distribuida 3 dormitorios amplios, 2 baños, un amplio y luminoso salón-comedor con salida a una terraza que te permitirá disfrutar de las mejores temporadas del año y una cocina independiente con salida a un balcón cerrado.~El piso ofrece la combinación perfecta de tranquilidad y proximidad a todos los servicios y comodidades a pocos minutos de distancia del centro de Madrid.~Inmueble sujeto a protección oficial.",NaN,None
30,"Piso en venta en La Latina, Madrid",https://www.pisosmadrid.com.es/propiedad/3874211-piso-en-venta-en-la-latina-madrid,La Latina,"PISO EN MADRID. ~Vivienda situada en la Calle Cardenal Mendoza, en el Barrio de la Latina, Madrid. VIVIENDA CON INCIDENCIAS: INMUEBLE SIN POSESIÓN. ~~El estado de ocupado y/o los impedimentos para que el inmueble pueda ser visitado libremente por los técnicos impiden su tasación oficial conforme exige la Orden ECO/805/2003, de 27 de marzo, por lo que el inmueble, mientras perdure la situación, no podrá ser hipotecado, con lo cual la compra sería al CONTADO. ~~Este inmueble ha sido adquirido por el Vendedor, pero aún no se ha tomado la posesión, al estar OCUPADO. El Adquirente asumirá; la situación física y estado de conservación; la situación urbanística; las discrepancias entre la realidad física y la realidad registral y catastral o en la cabida del Inmueble; la adecuación registral y catastral del Inmueble; y los tramites que conlleven la obtención de la Posesión Judicial del Inmueble y/o posibles acuerdos extrajudiciales con el ocupante en el caso de haberlo.~~Honorarios de gestión no incluidos en el precio de compra.~~VENDE SINERGIA INVESTMENT – 613 508 666 – 854 53 53 59~~www.inmobiliariasinergia.es",NaN,Cardenal
62,"Piso en venta en Vicalvaro, Madrid",https://www.pisosmadrid.com.es/propiedad/3890776-piso-en-venta-en-vicalvaro-madrid,Vicalvaro,"Piso 0 dormitorios, 0 baños, 0 garajes, Para reformar, en Madrid, Madrid inmueble sin posesion ocupado no es posible visita, se vende en sus condiciones Piso de 61 m2 situado en edificio de 5 plantas sobre rasante ubicado en la ciudad de Madrid provincia de Madrid y capital de España. Construido en el año 1960 no dispone de ascensor. Esta vivienda presenta una distribución que consta de salón comedor cocina dos dormitorios un baño y balcón. Se localiza en zona urbana tranquila y bien comunicada rodeada de viviendas de similares características con todos los servicios cercanos y accesibles",NaN,None
64,Piso en venta en Madrid,https://www.pisosmadrid.com.es/propiedad/3890778-piso-en-venta-en-madrid,NaN,"Piso 0 dormitorios, 0 baños, 0 garajes, Para reformar, en Madrid, Madrid Vivienda piso situado en un edificio residencial en la ciudad de Madrid. Edificio construido en el año 2005 compuesto de seis plantas sobre rasante ubicado en zona próxima al Real Jardín Botánico. La vivienda cuenta con una superficie construida de 30 m2 y útil de 26 m2. Ubicado en zona residencial con todo tipo de comercios y servicios próximos. La renta mensual actual de este inmueble es de 53,71, siendo un contrato de renta antigua. A fecha de publicación del presente anuncio el arrendatario se encuentra al corriente de pago de sus obligaciones siendo la renta estimada y la fecha de vencimiento (sin perjuicio de las prórrogas que conforme a ley correspondan) la que se hace constar en el anuncio. Las rentas devengadas hasta la fecha de transmisión corresponderán al vendedor y las que se devenguen a partir de la transmisión corresponderán al comprador.",NaN,None
68,"Piso en venta en Moratalaz, Madrid",https://www.pisosmadrid.com.es/propiedad/3890818-piso-en-venta-en-moratalaz-madrid,Moratalaz,"Piso 3 dormitorios, 1 baños, 0 garajes, Para reformar, en Madrid, Madrid Cuarto piso sin asce

In [1318]:
# nuda propiedad son imuebles sin posesion, se estan incluyendo inmuebles que se tiene la posesion pero no el uso y disfrute por que alguien tiene el usufructo, tabien se incluye inmuebles ocupados

In [1319]:
#elimino la calle si el barrio es nulo y vuelvo a calcular la calle cogiendo el valor hasta la coma-
dfpisosmadrid.loc[dfpisosmadrid["cod_barrio"].isna(), "calle"] = None


In [1320]:
dfpisosmadrid.loc[dfpisosmadrid["calle"].isnull(), "calle"] = dfpisosmadrid["descripcion_larga"].apply(extraer_calle)

In [1321]:
# lamar a la función pasándole el dataframe y el dataframe de referencia

clasificar_distrito_y_barrio_calle(dfpisosmadrid, dfcalles)

cardenalmendoza
cardenalmendoza
2
pinosalta
pinosalta
4
pinosalta
pinosalta
4
puertobalbaran
calleortegaygassetconocidaporsusboutiquesdealtacosturayjoyerias
plazasmasconocidasdelbarriodelalatina~~esteencantadorpisoseencuentradistribuidoen:ampliosalon-comedorconaccesoaungranpatiodeusoydisfrutedelapropiedad
plazadegarajeincluidaenelprecio.tambiensepuedeadquiriradicionalmenteunasegundaplazadegaraje.~~laconstrucciondelano2007
calle.comedorindependiente
degarajeincluidaenelprecio
degaraje.
nuestrasenoradevalvanera
nuestrasenoradevalvanera
0
juanboscan
juanboscan
2
callemenorca.muycercadelparquedelretiro.~~elcorteingles
plazadegaraje.~~actualmenteseencuentraalquilado
calledecastello
desanlamberto
calleclaudiocoelloa50m.escasosdej.ortegaygasset.~setratadeunaprimeraplantamuyluminosaapatiodemanzana.~susmasde100m2construidossedistribuyenenunbuensalon-comedorendosambientesdiferenciados
plazagarajeytrastero
plazacascorro.~~estamagnificaviviendaconstade5habitaciones
calle
plazadegarajeyexcelenteszo

,url,title,precio,dormitorios,banos,m2_constr,referencia,tipologia,planta,descripcion_larga,area,piscina,garaje,jardin,fecha_publicacion,num_visitas,calefaccion,trastero,amueblado,chimenea,videoportero,alarma,aire_acondicionado,barrio,cod_barrio,distrito,cod_distrito,codigo_Postal,nuda_propiedad,tiene_piscina,letrace,estado,gastos_comunidad,ascensor,ano_construccion,terraza,porteria,balcon,cocina,inmueble_ingresos,portero,exterior,armarios,plantas_edificio,lujo,orientacion_este,interior,orietacion_oeste,orientacion_sur,aire_ acondicionado,inmueble_ingresps,num_plantas,Lujo,calle
0,https://www.pisosmadrid.com.es/propiedad/3863841-piso-en-venta-en-madrid,Piso en venta en Madrid,60.000,3,1,54,3863841,Piso,NaN,"PISO EN MADRID. ~Vivienda situada en Avenida Palomeras, en Madrid. VIVIENDA CON INCIDENCIAS: INMUEBLE SIN POSESIÓN. ~~El estado de ocupado y/o los impedimentos para que el inmueble pueda ser visitado libremente por los técnicos impiden su tasación oficial conforme exige la Orden ECO/805/2003, de 27 de marzo, por lo que el inmueble, mientras perdure la situación, no podrá ser hipotecado, con lo cual la compra sería al CONTADO. ~~Este inmueble ha sido adquirido por el Vendedor, pero aún no se ha tomado la posesión, al estar OCUPADO. El Adquirente asumirá; la situación física y estado de conservación; la situación urbanística; las discrepancias entre la realidad física y la realidad registral y catastral o en la cabida del Inmueble; la adecuación registral y catastral del Inmueble; y los tramites que conlleven la obtención de la Posesión Judicial del Inmueble y/o posibles acuerdos extrajudiciales con el ocupante en el caso de haberlo.~~Honorarios de gestión no incluidos en el precio de compra.~~VENDE SINERGIA INVESTMENT – 613 508 666 – 854 53 53 59~~www.inmobiliariasinergia.es",NaN,0.0,0.0,0.0,2024-05-17,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PALOMERAS BAJAS,1,PUENTE DE VALLECAS,13,28018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Palomeras,"
1,https://www.pisosmadrid.com.es/propiedad/3865872-piso-en-venta-en-montecarmelo---el-goloso-madrid,"Piso en venta en Montecarmelo - El Goloso, Madrid",825.000,4,2,142,3865872,Piso,NaN,"600 Aprovecha esta oportunidad de compra de esta exclusiva vivienda de 4 espaciosos dormitorios en la mejor zona de Madrid y en comunidad privada con dos piscinas, una de ellas exclusiva para que disfruten los más pequeños. Con toda la carpintería renovada, la vivienda cambia el color marrón original en puertas y rodapiés por el blanco, ganando una luminosidad que se expande por toda la vivienda. Recién pintada cuenta también con ventanas oscilobatientes de máxima calidad recientemente instaladas, con rotura de puente térmico para aislar mejor la vivienda y ganar eficiencia energética, consiguiendo un ahorro en costes y mejora del medio ambiente. Las persianas eléctricas se acaban de incorporar a la vivienda, cada una con su propio mando, y además cuenta con uno universal para todas ellas (a excepción del dormitorio principal). El suelo de madera brilla recién barnizado. Qué maravilla poder disfrutar de esta vivienda de 4 dormitorios, probablemente de los más grandes y espaciosos de todo Montecarmelo. La cocina, completamente equipada, cuenta además con un amplio office para ubicar lavadora y secadora. Cuenta con aire acondicionado y bomba de calor, además de gas natural. Y por supuesto con una plaza de garaje y trastero para que puedas guardar las bicicletas que usarás por todos los km de carril bici que conecta Montecarmelo con el resto de Madrid. En la mejor zona de Montecarmelo, muy cerquita del centro comercial, con salida a una tranquila calle peatonal, ideal para niños, y a un paso del acceso a M30 y M40, cuenta además con transporte público y otros servicios como colegios públicos, concertados y privados, entre ellos el Alemán; varios supermercados, gimnasios, la magnífica Ciudad de la Raqueta y el Centro Nacional de Golf. Además en breve entre est

In [1322]:
dfpisosmadrid.query("barrio.isnull()")[["title","url","area","descripcion_larga","calle"]]

,title,url,area,descripcion_larga,calle
27,Piso en venta en Madrid,https://www.pisosmadrid.com.es/propiedad/3872239-piso-en-venta-en-madrid,NaN,"DP2020 vende piso en edificio residencial con ascensor, ubicado en la localidad de Madrid~Consta de una superficie de 102m² bien distribuida 3 dormitorios amplios, 2 baños, un amplio y luminoso salón-comedor con salida a una terraza que te permitirá disfrutar de las mejores temporadas del año y una cocina independiente con salida a un balcón cerrado.~El piso ofrece la combinación perfecta de tranquilidad y proximidad a todos los servicios y comodidades a pocos minutos de distancia del centro de Madrid.~Inmueble sujeto a protección oficial.",NaN
62,"Piso en venta en Vicalvaro, Madrid",https://www.pisosmadrid.com.es/propiedad/3890776-piso-en-venta-en-vicalvaro-madrid,Vicalvaro,"Piso 0 dormitorios, 0 baños, 0 garajes, Para reformar, en Madrid, Madrid inmueble sin posesion ocupado no es posible visita, se vende en sus condiciones Piso de 61 m2 situado en edificio de 5 plantas sobre rasante ubicado en la ciudad de Madrid provincia de Madrid y capital de España. Construido en el año 1960 no dispone de ascensor. Esta vivienda presenta una distribución que consta de salón comedor cocina dos dormitorios un baño y balcón. Se localiza en zona urbana tranquila y bien comunicada rodeada de viviendas de similares características con todos los servicios cercanos y accesibles",NaN
64,Piso en venta en Madrid,https://www.pisosmadrid.com.es/propiedad/3890778-piso-en-venta-en-madrid,NaN,"Piso 0 dormitorios, 0 baños, 0 garajes, Para reformar, en Madrid, Madrid Vivienda piso situado en un edificio residencial en la ciudad de Madrid. Edificio construido en el año 2005 compuesto de seis plantas sobre rasante ubicado en zona próxima al Real Jardín Botánico. La vivienda cuenta con una superficie construida de 30 m2 y útil de 26 m2. Ubicado en zona residencial con todo tipo de comercios y servicios próximos. La renta mensual actual de este inmueble es de 53,71, siendo un contrato de renta antigua. A fecha de publicación del presente anuncio el arrendatario se encuentra al corriente de pago de sus obligaciones siendo la renta estimada y la fecha de vencimiento (sin perjuicio de las prórrogas que conforme a ley correspondan) la que se hace constar en el anuncio. Las rentas devengadas hasta la fecha de transmisión corresponderán al vendedor y las que se devenguen a partir de la transmisión corresponderán al comprador.",NaN
68,"Piso en venta en Moratalaz, Madrid",https://www.pisosmadrid.com.es/propiedad/3890818-piso-en-venta-en-moratalaz-madrid,Moratalaz,"Piso 3 dormitorios, 1 baños, 0 garajes, Para reformar, en Madrid, Madrid Cuarto piso sin ascensor, distribuido en salón-comedor, cocina, tres dormitorios y un baño.",NaN
87,"Piso en venta en Puente de Vallecas, Madrid",https://www.pisosmadrid.com.es/propiedad/3895840-piso-en-venta-en-puente-de-vallecas-madrid,Puente de Vallecas,"PISO EN MADRID.~Vivienda situada en la Calle Puerto Balbaran, en Madrid. VIVIENDA CON INCIDENCIAS: INMUEBLE SIN POSESIÓN. ~~El estado de ocupado y/o los impedimentos para que el inmueble pueda ser visitado libremente por los técnicos impiden su tasación oficial conforme exige la Orden ECO/805/2003, de 27 de marzo, por lo que el inmueble, mientras perdure la situación, no podrá ser hipotecado, con lo cual la compra sería al CONTADO. ~~Este inmueble ha sido adquirido por el Vendedor, pero aún no se ha tomado la posesión, al estar OCUPADO. El Adquirente asumirá; la situación física y estado de conservación; la situación urbanística; las discrepancias entre la realidad física y la realidad registral y catastral o en la cabida del Inmueble; la adecuación registral y catastral del Inmueble; y los tramites que conlleven la obtención de la Posesión Judicial del Inmueble y/o posibles acuerdos extrajudiciales con el ocupante en el caso de haberlo.~~~VENDE SINERGIA INVESTMENT – 613 508 666 – 854 53 53 59~~www.inmobiliariasinergia.es",Calle Puerto Bal

In [1323]:
dfpisosmadrid["distrito"].value_counts(dropna=False)

distrito
Centro                  296
Salamanca               102
Carabanchel              46
Chamberí                 37
Chamartín                34
Puente de Vallecas       32
Arganzuela               31
Villaverde               30
Retiro                   30
Ciudad Lineal            29
Moncloa-Aravaca          28
Latina                   28
Tetuán                   25
 San Blas-Canillejas     21
Usera                    18
Fuencarral-El Pardo      14
Hortaleza                14
NaN                       7
 Villa de Vallecas        7
Moratalaz                 7
 Barajas                  5
Villa de Vallecas         5
TETUAN                    5
San Blas-Canillejas       4
SALAMANCA                 3
CENTRO                    2
LATINA                    2
CIUDAD LINEAL             1
CARABANCHEL               1
BARAJAS                   1
PUENTE DE VALLECAS        1
SAN BLAS-CANILLEJAS       1
VILLA DE VALLECAS         1
Barajas                   1
 Vicálvaro                1
VICALVARO  

In [1324]:
dfpisosmadrid["distrito"]=dfpisosmadrid["distrito"].str.lower().str.title()

In [1325]:
# Definir el diccionario de reemplazos
reemplazos = {
    'Chamberi': 'Chamberí',
    'Chamartin': 'Chamartín',
    'Tetuan': 'Tetuán',
    'Vicalvaro': 'Vicálvaro',
    'Fuencarral El Pardo':'Fuencarral-El Pardo'

}

# Reemplazar múltiples valores en la columna 'distrito'
dfpisosmadrid['distrito'] = dfpisosmadrid['distrito'].replace(reemplazos).str.strip()

In [1326]:
dfpisosmadrid.query("distrito.isnull()")

,url,title,precio,dormitorios,banos,m2_constr,referencia,tipologia,planta,descripcion_larga,area,piscina,garaje,jardin,fecha_publicacion,num_visitas,calefaccion,trastero,amueblado,chimenea,videoportero,alarma,aire_acondicionado,barrio,cod_barrio,distrito,cod_distrito,codigo_Postal,nuda_propiedad,tiene_piscina,letrace,estado,gastos_comunidad,ascensor,ano_construccion,terraza,porteria,balcon,cocina,inmueble_ingresos,portero,exterior,armarios,plantas_edificio,lujo,orientacion_este,interior,orietacion_oeste,orientacion_sur,aire_ acondicionado,inmueble_ingresps,num_plantas,Lujo,calle
64,https://www.pisosmadrid.com.es/propiedad/3890778-piso-en-venta-en-madrid,Piso en venta en Madrid,79.900,0,0,26,3890778,Piso,NaN,"Piso 0 dormitorios, 0 baños, 0 garajes, Para reformar, en Madrid, Madrid Vivienda piso situado en un edificio residencial en la ciudad de Madrid. Edificio construido en el año 2005 compuesto de seis plantas sobre rasante ubicado en zona próxima al Real Jardín Botánico. La vivienda cuenta con una superficie construida de 30 m2 y útil de 26 m2. Ubicado en zona residencial con todo tipo de comercios y servicios próximos. La renta mensual actual de este inmueble es de 53,71, siendo un contrato de renta antigua. A fecha de publicación del presente anuncio el arrendatario se encuentra al corriente de pago de sus obligaciones siendo la renta estimada y la fecha de vencimiento (sin perjuicio de las prórrogas que conforme a ley correspondan) la que se hace constar en el anuncio. Las rentas devengadas hasta la fecha de transmisión corresponderán al vendedor y las que se devenguen a partir de la transmisión corresponderán al comprador.",NaN,0.0,0.0,0.0,2024-06-10,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262,https://www.pisosmadrid.com.es/propiedad/3961341-piso-en-venta-en-madrid,Piso en venta en Madrid,138.600,1 dormitorio,1,109,3961341,Piso,NaN,"REF MAD-471 AKRIS INMOBILIARIA VENDE: Se presenta una oportunidad de inversiÃ³n en un piso ubicado en el corazÃ³n de Madrid. Con una superficie de 109.21 m2, este inmueble promete ser una opciÃ³n atractiva para inversores con visiÃ³n y disposiciÃ³n para enfrentar ciertos desafÃ­os. Actualmente, el piso se encuentra ocupado por una persona sin justo tÃ­tulo, lo que impide realizar visitas al lugar en este momento. Esta situaciÃ³n puede representar un obstÃ¡culo para algunos, pero para inversores experimentados, puede significar una oportunidad Ãºnica para adquirir una propiedad a un precio potencialmente reducido y aprovechar su potencial de valorizaciÃ³n a largo plazo. Para aquellos que estÃ©n interesados en esta oferta, AKRIS ofrece sus servicios para encontrar la vivienda adecuada y proporcionar el asesoramiento especializado necesario para asegurar la inversiÃ³n. Con AKRIS, los inversores pueden contar con un equipo comprometido y cercano que los guiarÃ¡ en cada paso del proceso. Si estÃ¡s buscando una inversiÃ³n desafiante pero potencialmente lucrativa en el mercado inmobiliario de Madrid, esta podrÃ­a ser la oportunidad que has estado esperando. ContÃ¡ctanos ahora para obtener mÃ¡s informaciÃ³n y dar el primer paso hacia tu prÃ³xima inversiÃ³n exitosa.",NaN,0.0,0.0,0.0,2024-07-22,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
270,https://www.pisosmadrid.com.es/propiedad/3963177-piso-en-venta-en-arturo-soria-madrid,"Piso en venta en Arturo Soria, Madrid",450.410,1 dormitorio,1,70,3963177,Piso,NaN,"Exclusiva zona y residencial privado con porteria, vivienda de 1 dormitorio con posibilidad de 2, 70 m2, amplia terraza. Urbanización con piscina. Plaza de garaje.",Arturo Soria,1.0,1.0,0.0,2024-07-23,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Plaza de garaje.
361,htt

**_La funcion de arriba no genero cambio  pero ahora se que no tengo que hacerla mas adelante, asi que no la eliminare..... dejo este nota aqui_**

In [1327]:
dfpisosmadrid["distrito"].isna().sum()

7

In [1328]:
dfpisosmadrid["barrio"].isna().sum()

49

In [1329]:
dfpisosmadrid.shape

(873, 54)

In [1330]:
dfpisosmadrid["barrio"].isna().sum()

49

In [1331]:
dfpisosmadrid.duplicated().sum()

0

In [1332]:
dfpisosmadrid.isna().sum()

url                      0
title                    0
precio                   0
dormitorios              0
banos                    0
m2_constr                0
referencia               0
tipologia                0
planta                 723
descripcion_larga       15
area                   126
piscina                 15
garaje                  15
jardin                  15
fecha_publicacion       15
num_visitas             15
calefaccion            297
trastero               690
amueblado              714
chimenea               869
videoportero           591
alarma                 828
aire_acondicionado     830
barrio                  49
cod_barrio              49
distrito                 7
cod_distrito             7
codigo_Postal          103
nuda_propiedad         865
tiene_piscina          867
letrace                871
estado                 851
gastos_comunidad       869
ascensor               866
ano_construccion       867
terraza                859
porteria               864
b

In [1333]:
dfpisosmadrid.query("cod_barrio.isnull()")[["url","barrio","distrito","tipologia","calle","area"]]

,url,barrio,distrito,tipologia,calle,area
27,https://www.pisosmadrid.com.es/propiedad/3872239-piso-en-venta-en-madrid,NaN,Centro,Piso,NaN,NaN
62,https://www.pisosmadrid.com.es/propiedad/3890776-piso-en-venta-en-vicalvaro-madrid,NaN,Vicálvaro,Piso,NaN,Vicalvaro
64,https://www.pisosmadrid.com.es/propiedad/3890778-piso-en-venta-en-madrid,NaN,NaN,Piso,NaN,NaN
68,https://www.pisosmadrid.com.es/propiedad/3890818-piso-en-venta-en-moratalaz-madrid,NaN,Moratalaz,Piso,NaN,Moratalaz
87,https://www.pisosmadrid.com.es/propiedad/3895840-piso-en-venta-en-puente-de-vallecas-madrid,NaN,Puente De Vallecas,Piso,Calle Puerto Balbaran,Puente de Vallecas
120,https://www.pisosmadrid.com.es/propiedad/3912810-piso-en-venta-en-salamanca-madrid,NaN,Salamanca,Piso,CALLE ORTEGA Y GASSET conocida por sus boutiques de alta costura y joyerías,Salamanca
145,https://www.pisosmadrid.com.es/propiedad/3920110-piso-en-venta-en-tetuan-madrid,NaN,Tetuán,Piso,NaN,Tetuan
174,https://www.pisosmadrid.com.es/propiedad/3930160-piso-en-venta-en-puente-de-vallecas-madrid,NaN,Puente De Vallecas,Piso,NaN,Puente de Vallecas
180,https://www.pisosmadrid.com.es/propiedad/3931856-piso-en-venta-en-la-latina-madrid,NaN,Latina,Piso,plazas más conocidas del Barrio de La Latina~~Este encantador piso se encuentra distribuido en: amplio salón-comedor con acceso a un gran patio de uso y disfrute de la propiedad,La Latina
222,https://www.pisosmadrid.com.es/propiedad/3948738-piso-en-venta-en-ciudad-lineal-madrid,NaN,Ciudad Lineal,Piso,NaN,Ciudad Lineal


In [1334]:
pd.set_option('display.max_colwidth', None)

In [1335]:
dfpisosmadrid['web']="pisosmadrid"
dfpisosmadrid['fecha_descarga']="24/10/2024"

In [1336]:
# Mapeo de valores textuales a valores numéricos
mapeo_plantas = {
    "primera baja": 0, "Bajo": 0, "Sótano": -1, "sotano": -1,
    "primera planta": 1, "Primera planta": 1, "1ª planta": 1, "1a planta": 1,
    "segunda planta": 2, "Segunda planta": 2, "2ª planta": 2, "2a planta": 2,
    "tercera planta": 3, "Tercera planta": 3, "3ª planta": 3, "3a planta": 3,
    "cuarta planta": 4, "Cuarta planta": 4, "4ª planta": 4, "4a planta": 4,
    "quinta planta": 5, "Quinta planta": 5, "5ª planta": 5, "5a planta": 5,
    "sexta planta": 6, "Sexta planta": 6, "6ª planta": 6, "6a planta": 6,
    "séptima planta": 7, "Séptima planta": 7, "7ª planta": 7, "7a planta": 7,
    "octava planta": 8, "Octava planta": 8, "8ª planta": 8, "8a planta": 8,
    "novena planta": 9, "Novena planta": 9, "9ª planta": 9, "9a planta": 9,
    "décima planta": 10, "Décima planta": 10, "10ª planta": 10, "10a planta": 10,
    "undécima planta": 11, "Undécima planta": 11, "11ª planta": 11, "11a planta": 11,
    "duodécima planta": 12, "Duodécima planta": 12, "12ª planta": 12, "12a planta": 12,
    "decimotercera planta": 13, "Decimotercera planta": 13, "13ª planta": 13, "13a planta": 13,
    "decimocuarta planta": 14, "Decimocuarta planta": 14, "14ª planta": 14, "14a planta": 14,
    "decimoquinta planta": 15, "Decimoquinta planta": 15, "15ª planta": 15, "15a planta": 15,
    "decimosexta planta": 16, "Decimosexta planta": 16, "16ª planta": 16, "16a planta": 16,
    "decimoséptima planta": 17, "Decimoséptima planta": 17, "17ª planta": 17, "17a planta": 17,
    "decimoctava planta": 18, "Decimoctava planta": 18, "18ª planta": 18, "18a planta": 18,
    "decimonovena planta": 19, "Decimonovena planta": 19, "19ª planta": 19, "19a planta": 19,
    "vigésima planta": 20, "Vigésima planta": 20, "20ª planta": 20, "20a planta": 20
}

In [1337]:
dfpisosmadrid.query("precio.isnull()")

,url,title,precio,dormitorios,banos,m2_constr,referencia,tipologia,planta,descripcion_larga,area,piscina,garaje,jardin,fecha_publicacion,num_visitas,calefaccion,trastero,amueblado,chimenea,videoportero,alarma,aire_acondicionado,barrio,cod_barrio,distrito,cod_distrito,codigo_Postal,nuda_propiedad,tiene_piscina,letrace,estado,gastos_comunidad,ascensor,ano_construccion,terraza,porteria,balcon,cocina,inmueble_ingresos,portero,exterior,armarios,plantas_edificio,lujo,orientacion_este,interior,orietacion_oeste,orientacion_sur,aire_ acondicionado,inmueble_ingresps,num_plantas,Lujo,calle,web,fecha_descarga


In [1338]:
#dfpisosmadrid["precio"] = dfpisosmadrid["precio"].str.strip()
dfpisosmadrid["precio"] = dfpisosmadrid["precio"].astype(str).str.strip()
dfpisosmadrid["precio"] = dfpisosmadrid["precio"].str.replace(".", "", regex=False)
dfpisosmadrid["precio"] = dfpisosmadrid["precio"].astype(int)

In [1342]:
dfpisosmadrid.query("url=='https://www.pisosmadrid.com.es/propiedad/3890818-piso-en-venta-en-moratalaz-madrid'")

,url,title,precio,dormitorios,banos,m2_constr,referencia,tipologia,planta,descripcion_larga,area,piscina,garaje,jardin,fecha_publicacion,num_visitas,calefaccion,trastero,amueblado,chimenea,videoportero,alarma,aire_acondicionado,barrio,cod_barrio,distrito,cod_distrito,codigo_Postal,nuda_propiedad,tiene_piscina,letrace,estado,gastos_comunidad,ascensor,ano_construccion,terraza,porteria,balcon,cocina,inmueble_ingresos,portero,exterior,armarios,plantas_edificio,lujo,orientacion_este,interior,orietacion_oeste,orientacion_sur,aire_ acondicionado,inmueble_ingresps,num_plantas,Lujo,calle,web,fecha_descarga
68,https://www.pisosmadrid.com.es/propiedad/3890818-piso-en-venta-en-moratalaz-madrid,"Piso en venta en Moratalaz, Madrid",198000,3,1,69,3890818,Piso,NaN,"Piso 3 dormitorios, 1 baños, 0 garajes, Para reformar, en Madrid, Madrid Cuarto piso sin ascensor, distribuido en salón-comedor, cocina, tres dormitorios y un baño.",Moratalaz,0.0,0.0,0.0,2024-06-10,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Moratalaz,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pisosmadrid,24/10/2024


In [1339]:
# Sustituir los valores textuales por numéricos en la columna "planta"
dfpisosmadrid["planta"] = dfpisosmadrid["planta"].map(mapeo_plantas).fillna(dfpisosmadrid["planta"])

In [1340]:
#dormitorios
dfpisosmadrid["dormitorios"]=dfpisosmadrid["dormitorios"].str.replace("1 dormitorio","1")

In [1341]:
dfpisosmadrid.to_csv("dfpisosmadrid_paraunir.csv",sep=";",index="False")